<a href="https://colab.research.google.com/github/arjunsinghrathore/Subject-Independent-Emotion-Recognition/blob/main/AE_LFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Jun  7 05:29:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **AE TRAINING IND**

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf

batch_size = 500
original_dim = 32
epochs = 15
zscore = True

for subNo in range(1,33): #subNo 从1到32
    for latent_dim in range(128,129):
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))
        # encoder
        eeg_input = Input(shape=(original_dim,))
        dense1 = Dense(latent_dim, activation='sigmoid')(eeg_input)
        encoder = Model(eeg_input, dense1)
        # decoder
        eeg_output = Dense(original_dim)(dense1)

        ae = Model(eeg_input, eeg_output)
        ae.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        x_train = np.transpose(x_train)[:, 0:32]
        x_test = x_train

        print('x_train shape : ',x_train.shape)

        ae.fit(x_train, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test, x_test))

        # build a model to project inputs
        decoded_x = ae.predict(x_train)
        encoded_x = encoder.predict(x_train)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/AE/encoded_eegs_1ae/encoded_eegs_1ae_sub' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/VAE/encoded_eegs_2vae_z_sub/encoded_eegs_2vae_z_sub' +
        #             str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #             {'encoded_eegs_z': encoded_x_z})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        #sio.savemat('D:\\VAE Experiment\\DEAP\\decoded_eegs_2vae\\decoded_eegs_2vae_sub' +
        #            str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #            {'decoded_eegs': decoded_x})

# **DEEP AE IND**

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf

batch_size = 500
original_dim = 32
epochs = 30
zscore = True

for subNo in range(1,33): #subNo 从1到32
    for latent_dim in range(32,33):
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))
        # encoder
        eeg_input = Input(shape=(original_dim,))
        encoded = Dense(latent_dim*4, activation='sigmoid')(eeg_input)
        encoded = Dense(latent_dim*2, activation='sigmoid')(encoded)
        dense1 = Dense(latent_dim, activation='sigmoid')(encoded)
        encoder = Model(eeg_input, dense1)
        # decoder
        decoded = Dense(latent_dim*2, activation='sigmoid')(dense1)
        decoded = Dense(latent_dim*4, activation='sigmoid')(decoded)
        eeg_output = Dense(original_dim)(decoded)

        ae = Model(eeg_input, eeg_output)
        ae.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        x_train = np.transpose(x_train)[:, 0:32]
        x_test = x_train

        print('x_train shape : ',x_train.shape)

        ae.fit(x_train, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test, x_test))

        # build a model to project inputs
        decoded_x = ae.predict(x_train)
        encoded_x = encoder.predict(x_train)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DAE/encoded_eegs_1Dae/encoded_eegs_1Dae_sub' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/VAE/encoded_eegs_2vae_z_sub/encoded_eegs_2vae_z_sub' +
        #             str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #             {'encoded_eegs_z': encoded_x_z})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        #sio.savemat('D:\\VAE Experiment\\DEAP\\decoded_eegs_2vae\\decoded_eegs_2vae_sub' +
        #            str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #            {'decoded_eegs': decoded_x})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=32
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DAE/encoded_eegs_1Dae/encoded_eegs_1Dae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DAE_ecoded_dataset_32.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

# **Convolutional 1D AE**

In [ ]:
# from tqdm import tqdm
# import numpy as np
# from keras.layers import Input, Dense, LeakyReLU, Conv1D, MaxPooling1D, UpSampling1D, Reshape, Flatten, AveragePooling1D
# from keras import layers
# from keras.models import Model
# import scipy.io as sio
# import h5py
# import tensorflow as tf

# batch_size = 500
# original_dim = 32
# epochs = 30
# zscore = True
# latent_dim = 16

# input_window = Input(shape=(original_dim,1))
# print(input_window.shape)
# x = Conv1D(16, 3, activation="relu", padding="same")(input_window) # 10 dims
# #x = BatchNormalization()(x)
# x = MaxPooling1D(2, padding="same")(x) # 5 dims
# x = Conv1D(1, 3, activation="relu", padding="same")(x) # 5 dims
# #x = BatchNormalization()(x)
# encoded = MaxPooling1D(1, padding="same")(x) # 3 dims
# print(encoded.shape)
# encoded = AveragePooling1D()(encoded)
# print(encoded.shape)
# encoded = attention()(encoded) # this is added

# encoded = Flatten()(encoded)
# encoded = Dense(latent_dim, activation='relu')(encoded)
# print(encoded.shape)
# encoder = Model(input_window, encoded)

# # 3 dimensions in the encoded layer
# x = Dense(latent_dim)(encoded)
# x = Reshape((16,1))(x)
# x = Conv1D(1, 3, activation="relu", padding="same")(x) # 3 dims
# #x = BatchNormalization()(x)
# x = UpSampling1D(2)(x) # 6 dims
# x = Conv1D(16, 2, activation='relu', padding="same")(x) # 5 dims
# #x = BatchNormalization()(x)
# x = UpSampling1D(1)(x) # 10 dims
# decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x) # 10 dims
# print(decoded.shape)


In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape, AveragePooling1D, BatchNormalization
from keras import layers
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf

batch_size = 500
original_dim = 32
epochs = 15
zscore = True
SNR = 1

for subNo in range(1,33): #subNo 从1到32
    for latent_dim in range(8,9):
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))
        input_window = Input(shape=(original_dim,1))
        x = Conv1D(16, 3, activation="relu", padding="same")(input_window) # 10 dims
        x = BatchNormalization()(x)
        x = MaxPooling1D(2, padding="same")(x) # 5 dims
        x = Conv1D(1, 3, activation="relu", padding="same")(x) # 5 dims
        x = BatchNormalization()(x)
        encoded = MaxPooling1D(2, padding="same")(x) # 3 dims
        # encoded = AveragePooling1D()(encoded)
        #encoded = attention()(encoded) # this is added

        encoded = Flatten()(encoded)
        encoded = Dense(latent_dim, activation='softmax')(encoded)

        encoder = Model(input_window, encoded)

        # 3 dimensions in the encoded layer
        x = Dense(latent_dim)(encoded)
        x = Reshape((8,1))(x)
        x = Conv1D(1, 3, activation="relu", padding="same")(x) # 3 dims
        x = BatchNormalization()(x)
        x = UpSampling1D(2)(x) # 6 dims
        x = Conv1D(16, 2, activation='relu', padding="same")(x) # 5 dims
        x = BatchNormalization()(x)
        x = UpSampling1D(2)(x) # 10 dims
        decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x) # 10 dims
        
        ae = Model(input_window, decoded)
        ae.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']

        x_train = np.transpose(x_train)[:, 0:32]

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,32)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,32,1)
        x_train_noise = x_train_noise.reshape(-1,32,1)
        
        x_test = x_train
        x_test_noise = x_train_noise

        print('x_train shape : ',x_train.shape)

        ae.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test_noise, x_test))

        # build a model to project inputs
        decoded_x = ae.predict(x_train)
        encoded_x = encoder.predict(x_train)
        encoded_x = encoded_x.reshape(-1,latent_dim)

        print('encoded_x : ',encoded_x.shape)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/CNNAE/encoded_eegs_1CNNae/encoded_eegs_1CNNae_sub2222' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/VAE/encoded_eegs_2vae_z_sub/encoded_eegs_2vae_z_sub' +
        #             str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #             {'encoded_eegs_z': encoded_x_z})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        #sio.savemat('D:\\VAE Experiment\\DEAP\\decoded_eegs_2vae\\decoded_eegs_2vae_sub' +
        #            str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #            {'decoded_eegs': decoded_x})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/CNNAE/encoded_eegs_1CNNae/encoded_eegs_1CNNae_sub2222'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DCNNAE_ecoded_dataset_8888.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

# **Contractive AE IND**

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

batch_size = 500
original_dim = 32
epochs = 15
zscore = True




for subNo in range(1,33): #subNo 从1到32
    for latent_dim in range(32,33):

        lam = 1e-4

        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))
        # encoder
        eeg_input = Input(shape=(original_dim,))
        dense1 = Dense(latent_dim, activation='sigmoid', name='encoded')(eeg_input)
        encoder = Model(eeg_input, dense1)
        # decoder
        eeg_output = Dense(original_dim)(dense1)
        

        ae = Model(eeg_input, eeg_output)

        # def contractive_loss(y_pred, y_true):
        #       mse = K.mean(K.square(y_true - y_pred), axis=1)

        #       W = K.variable(value=ae.get_layer('encoded').get_weights()[0])  # N x N_hidden
        #       W = K.transpose(W)  # N_hidden x N
        #       h = ae.get_layer('encoded').output
        #       dh = h * (1 - h)  # N_batch x N_hidden

        #       # N_batch x N_hidden * N_hidden x 1 = N_batch x 1
        #       contractive = lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)

        #       return mse + contractive


        mse = K.mean(K.square(eeg_input - eeg_output), axis=1)
        W = K.variable(value=ae.get_layer('encoded').get_weights()[0])  # N x N_hidden
        W = K.transpose(W)  # N_hidden x N
        h = ae.get_layer('encoded').output

        dh = h * (1 - h)  # N_batch x N_hidden

        # N_batch x N_hidden * N_hidden x 1 = N_batch x 1
        contractive = lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)

        ae_loss = mse + contractive
        ae.add_loss(ae_loss)
        

        ae.compile(optimizer='adam')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        x_train = np.transpose(x_train)[:, 0:32]
        x_test = x_train

        print('x_train shape : ',x_train.shape)

        ae.fit(x_train, x_train,
                #shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test, x_test))

        # build a model to project inputs
        decoded_x = ae.predict(x_train)
        encoded_x = encoder.predict(x_train)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/CAE/encoded_eegs_1Cae/encoded_eegs_1Cae_sub' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/VAE/encoded_eegs_2vae_z_sub/encoded_eegs_2vae_z_sub' +
        #             str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #             {'encoded_eegs_z': encoded_x_z})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        #sio.savemat('D:\\VAE Experiment\\DEAP\\decoded_eegs_2vae\\decoded_eegs_2vae_sub' +
        #            str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #            {'decoded_eegs': decoded_x})

# **Seq-Seq LSTM AE IND**

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf

batch_size = 512
original_dim = 32
epochs = 20
zscore = True

for subNo in range(1,33): #subNo 从1到32

        timesteps = 1  # Length of your sequences
        input_dim = 32 
        latent_dim = 16
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        encoded = LSTM(latent_dim)(inputs)

        decoded = RepeatVector(timesteps)(encoded)
        decoded = LSTM(input_dim, return_sequences=True)(decoded)

        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)
        sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        x_train = np.transpose(x_train)[:, 0:32]
        x_train = x_train.reshape(-1,1,32)
        x_test = x_train

        print('x_train shape : ',x_train.shape)

        sequence_autoencoder.fit(x_train, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test, x_test))

        # build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/SEQAE/encoded_eegs_1SEQae/encoded_eegs_1SEQae_sub' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/VAE/encoded_eegs_2vae_z_sub/encoded_eegs_2vae_z_sub' +
        #             str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #             {'encoded_eegs_z': encoded_x_z})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        #sio.savemat('D:\\VAE Experiment\\DEAP\\decoded_eegs_2vae\\decoded_eegs_2vae_sub' +
        #            str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #            {'decoded_eegs': decoded_x})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=16
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/SEQAE/encoded_eegs_1SEQae/encoded_eegs_1SEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/SEQAE_ecoded_dataset_16.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

# **DeNoising Seq-Sec AE IND**

In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

batch_size = 512
original_dim = 32
epochs = 25
zscore = True
SNR = 1

for subNo in range(1,33): #subNo 从1到32

        timesteps = 1  # Length of your sequences
        input_dim = 32 
        latent_dim = 8
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        encoded = LSTM(latent_dim*2)(inputs)
        encoded = LSTM(latent_dim)(encoded)

        # decoded = RepeatVector(timesteps)(encoded)
        decoded = LSTM(latent_dim*2)(decoded)
        decoded = LSTM(input_dim, return_sequences=True)(decoded)

        # inputs = Input(shape=(timesteps, input_dim))
        # encoded = LSTM(latent_dim, activation="relu", return_sequences=False)(inputs)
        # #encoded = attention()(encoded) # this is added


        # decoded = RepeatVector(timesteps, name="bottleneck_output")(encoded)
        # decoded = LSTM(latent_dim, activation="relu", return_sequences=True)(decoded)
        # decoded = TimeDistributed(Dense(input_dim))(decoded)

        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)
        sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        x_train = np.transpose(x_train)[:, 0:32]

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,32)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,1,32)
        x_train_noise = x_train_noise.reshape(-1,1,32)
        
        x_test = x_train
        x_test_noise = x_train_noise

        print('x_train shape : ',x_train.shape)

        sequence_autoencoder.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test_noise, x_test))

        # build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub888888_' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub' +
        #         str(subNo) + '_latentdim' + str(Config['latent_dim']) + '.mat',
        #          {'decoded_eegs': decoded_x.reshape(-1,32)})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub888888_'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nz.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['decoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_decoded_dataset_64.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

# **DeNoising Seq-Sec AE IND T**

In [ ]:
import math
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer, LeakyReLU, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape, GlobalAveragePooling1D, Bidirectional
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return output

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
# from tqdm import tqdm
# import numpy as np
# from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector
# from keras.models import Model
# import scipy.io as sio
# import h5py
# import tensorflow as tf
# import keras.backend as K

# batch_size = 512
# original_dim = 32
# zscore = True

# X_train = []

# for subNo in tqdm(range(1,33)): #subNo 从1到32
#         #### train the VAE on normalized (z-score) multi-channel EEG data
#         print('subNo: '+str(subNo))
#         if zscore:
#             sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
#             x_train = sub_data_file['zscore_data']
#         else:
#             sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
#             x_train = sub_data_file['minmax_data']

#         print('shape : ',x_train.shape)
#         X_train.append(np.transpose(x_train)[:, 0:32])




In [ ]:
# X_train = np.array(X_train).reshape(-1,8064,32)
# X_train.shape

In [ ]:
batch_size = 40
original_dim = 32
epochs = 600
zscore = True
SNR = 1

for subNo in range(1,2): #subNo 从1到32

        timesteps = 8064  # Length of your sequences
        input_dim = 32 
        latent_dim = 8
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        encoded = LSTM(latent_dim, return_sequences=True)(inputs)
        encoded = attention()(tf.transpose(encoded, perm = [0,2,1]))
        encoded = tf.transpose(encoded, perm = [0,2,1])
        
        decoded = LSTM(input_dim, return_sequences=True)(encoded)
        

        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)
        sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        x_train = np.transpose(x_train)[:, 0:32]
        x_train = x_train.reshape(-1,8064,32)
        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,8064,32)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,8064,32)
        x_train_noise = x_train_noise.reshape(-1,8064,32)
        
        x_test = x_train
        x_test_noise = x_train_noise

        print('x_train shape : ',x_train.shape)

        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        callbacks = [es]

        sequence_autoencoder.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                verbose = 1,
                validation_data=(x_test_noise, x_test),
                callbacks = callbacks)

        #build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)

        print('encoded shape : ', encoded_x.shape)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub888888z_7680nrrrr_c_attn44_bino' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})


        # print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))

In [ ]:
 sequence_autoencoder.summary()

## **Visualization**

In [ ]:
!pip install mne

In [ ]:
import scipy.io as sio
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import h5py
import scipy.io as sio
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import mne
# from cuml.manifold import TSNE
%matplotlib inline

In [ ]:
# units = int(int(sequence_autoencoder.layers[5].trainable_weights[0].shape[1])/4)
# print("No units: ", units)

In [ ]:
# W = sequence_autoencoder.layers[5].get_weights()[0]
# U = sequence_autoencoder.layers[5].get_weights()[1]
# b = sequence_autoencoder.layers[5].get_weights()[2]

# W_i = W[:, :units]
# W_f = W[:, units: units * 2]
# W_c = W[:, units * 2: units * 3]
# W_o = W[:, units * 3:]

# U_i = U[:, :units]
# U_f = U[:, units: units * 2]
# U_c = U[:, units * 2: units * 3]
# U_o = U[:, units * 3:]

# b_i = b[:units]
# b_f = b[units: units * 2]
# b_c = b[units * 2: units * 3]
# b_o = b[units * 3:]

In [ ]:
# (sorted(W_i[4]+W_f[4]+W_c[4]))[::-1]

In [ ]:
# xxx=W_i[4]+W_f[4]+W_c[4]
# xxx[8]

In [ ]:
sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(10) + '.mat', 'r')
x_train = sub_data_file['zscore_data']
x_train = np.transpose(x_train)[:, 0:32]
x_train = x_train.reshape(-1,8064,32)
x_train.shape

# import _pickle as cPickle
# filename = '/content/drive/My Drive/EMOTION/data_preprocessed_python/s01.dat'
# x_train = (cPickle.load(open(filename, 'rb'), encoding='latin1'))['data'][:, :32, :]
# x_train.shape

In [ ]:
import pickle
with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nzz7680_attn444__l.pkl', 'rb') as filepath:
          x_data = pickle.load(filepath)

x_data = x_data.reshape(32,40,8064,-1)
x_data.shape

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

x_data = ZscoreNormalization(x_data)

In [ ]:
ind = 26

In [ ]:
xxx = x_data[9, ind]
print(xxx.shape)

In [ ]:
from scipy import signal
def ave_freq(data, freq = 128):
    win = 4 * freq
    freqs, psd = signal.welch(data, freq, nperseg=win, scaling='spectrum')
    print(psd.shape)
    #print(freqs[4:160])
    return psd[:,:].mean(1)

dataa_lt = ave_freq(np.transpose(xxx))

In [ ]:
dataa_lt

In [ ]:
np.argsort(np.abs(dataa_lt))[::-1]

In [ ]:
# Write a function that returns the cosine similarity based on the formula above
def cosine_similarity(vectorA,vectorB):
    return np.dot(vectorA,vectorB)/(np.linalg.norm(vectorA)*np.linalg.norm(vectorB))

In [ ]:
def most_similar(custom_vector, embeddings_dict):
  # We find the cosine similarity values with every other vector in the embeddings dictionary
  cosine_values = {i:cosine_similarity(v,custom_vector) for i, v in enumerate(embeddings_dict)}
  # We now find the embedding that is closest
  closest = sorted(cosine_values.items(),key =lambda item : item[1], reverse=True)

  return cosine_values, closest[:10]

In [ ]:
X = np.transpose(x_train[ind])
X.shape

In [ ]:
cos_val, clos = most_similar(xxx[:,2], X)

In [ ]:
# array([ 5, 24, 25, 30,  8,  0,  9,  6,  7, 28, 15,  2,  4, 11, 14, 12, 17,
#        23, 19,  3,  1, 16, 22, 10, 18, 27, 13, 26, 31, 21, 20, 29])

In [ ]:
cos_val

In [ ]:
clos

In [ ]:
ch_names = ['Fp1'	
        ,'AF3'
        ,'F3'
        ,'F7'
        ,'FC5'
        ,'FC1'
        ,'C3'
        ,'T7'
        ,'CP5'
        ,'CP1'
        ,'P3'
        ,'P7'
        ,'PO3'
        ,'O1'
        ,'Oz'
        ,'Pz'
        ,'Fp2'
        ,'AF4'
        ,'Fz'
        ,'F4'
        ,'F8'
        ,'FC6'
        ,'FC2'
        ,'Cz'
        ,'C4'
        ,'T8'
        ,'CP6'
        ,'CP2'
        ,'P4'
        ,'P8'
        ,'PO4'
        ,'O2']

In [ ]:
len(ch_names)

In [ ]:
import _pickle as cPickle
filename = '/content/drive/My Drive/EMOTION/data_preprocessed_python/s25.dat'
X = (cPickle.load(open(filename, 'rb'), encoding='latin1'))['data'][:, :32, :]
X.shape

In [ ]:
# X = np.transpose(x_train[ind])

In [ ]:
def mne_object(data, ch_names, freq, events = None):
  # create an mne info file with meta data about the EEG
  info = mne.create_info(ch_names=list(ch_names), 
                         sfreq=freq, 
                         ch_types=['eeg']*data.shape[0])
  
  # data needs to be in volts rather than in microvolts
  data = data*1e-6
  # transpose the data
  data_T = data#.transpose()
  
  # create raw mne object
  raw = mne.io.RawArray(data_T, info)

  if events:
    start_times = np.array(events[::2])
    end_times = np.array(events[1::2])
    anno_length = end_times-start_times
    event_name = np.array(['Ictal']*len(anno_length))

    raw.set_annotations(mne.Annotations(start_times,
                                      anno_length,
                                      event_name))

  return raw

mne_data = mne_object(X[26], ch_names, 128)

In [ ]:
# mne_data.info['ch_names']

In [ ]:
# set the standard montage
mne_data.set_montage('standard_1020')

In [ ]:
mne_data.plot_sensors(kind='topomap', show_names=True, to_sphere=True);
fig = mne_data.plot_sensors(kind='3d', show_names=True, show=False)
fig = fig.gca().view_init(azim=70, elev=15)
plt.show()

In [ ]:
mne_data[:][0].shape

In [ ]:
from scipy import signal
def ave_freq(data, freq = 128):
    win = 4 * freq
    freqs, psd = signal.welch(data, freq, nperseg=win, scaling='spectrum')
    print(psd.shape)
    #print(freqs[4:160])
    return psd[:,:].mean(1)

dataa = ave_freq(mne_data[:][0])

In [ ]:
np.argsort(np.abs(dataa))[::-1]

In [ ]:
mne.viz.plot_topomap(dataa,
                      mne_data.info,
                      show=False,
                      sensors=False,
                      names=mne_data.info['ch_names'], 
                      show_names=True)

In [ ]:
ch_names.index('F7')

In [ ]:
ch_names.index('C4')

In [ ]:
ch_names.index('P3')

In [ ]:
ch_names.index('P4')

In [ ]:
print(ch_names[24])

In [ ]:
print(ch_names[10])
print(ch_names[13])
print(ch_names[30:])

## **Post-Processing**

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub888888z_7680nrrrr_c_attn44_bino'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = trainSubData.reshape(-1,8)
        # print(traineSubData.shape)
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_final.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train), filepath, protocol=4)

In [ ]:
np.array(X_train).shape

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['decoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_decoded_dataset_64.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

# PCA  

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/pca/encoded_eegs_pca_sub'+ str(trainSubNo) + '_latedtdim' + str(latdim) + '.mat', 'r')
        trainSubData = np.array(file2['encoded_eegs'])
        trainSubData = trainSubData.reshape(-1,8)
        # print(traineSubData.shape)
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_pca.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train), filepath, protocol=4)

In [ ]:
np.array(X_train).shape

# ICA DEAP
  

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

X_train = []

for trainSubNo in range(1,33):
        file2 = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/ICA/encoded_eegs_ica_sub'+ str(trainSubNo) + '_latedtdim' + str(latdim) + '.mat', 'r')
        trainSubData = np.array(file2['encoded_eegs'])
        trainSubData = trainSubData.reshape(-1,8)
        # print(traineSubData.shape)
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_ICA.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train), filepath, protocol=4)

In [ ]:
np.array(X_train).shape

# **DeNoising Seq-Sec AE IND SEED**

In [ ]:
import math
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K
import os
import re

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
def get_all_purp(root_dir):
   all_mat = dict()
   for dirName, _, fileList in os.walk(root_dir):
       for fname in fileList[:-1]:
           #print(fname)
           if '.mat' in fname:
              sub_no = int(re.split(r'/|_',fname)[-2])
              if sub_no in all_mat:
                all_exist = all_mat[sub_no]
                all_mat[sub_no] = all_exist + [dirName + '/' + fname]
              else:
                all_mat[sub_no] = [dirName + '/' + fname]
   return all_mat

In [ ]:
all_files = get_all_purp('/content/drive/MyDrive/EMOTION/Preprocessed_EEG')

In [ ]:
all_files[1]

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

batch_size = 256
original_dim = 62
epochs = 5
SNR = 1

for subNo in range(1,16): #subNo 从1到32

        timesteps = 1  # Length of your sequences
        input_dim = 62 
        latent_dim = 16
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        encoded = LSTM(latent_dim)(inputs)

        decoded = RepeatVector(timesteps)(encoded)
        decoded = LSTM(input_dim, return_sequences=True)(decoded)

        # inputs = Input(shape=(timesteps, input_dim))
        # encoded = LSTM(latent_dim, activation="relu", return_sequences=False)(inputs)
        # #encoded = attention()(encoded) # this is added


        # decoded = RepeatVector(timesteps, name="bottleneck_output")(encoded)
        # decoded = LSTM(latent_dim, activation="relu", return_sequences=True)(decoded)
        # decoded = TimeDistributed(Dense(input_dim))(decoded)

        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)
        sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        
        x_train = np.zeros((15,30000,62))

        sub_data_file = []

        for filee in all_files[subNo]:
          temp = sio.loadmat(filee)
          temp = list(temp.values())[3:]
          sub_data_file = sub_data_file + temp
          break

        print('Sub Data File : ',len(sub_data_file))

        
        for i, dataa in enumerate(sub_data_file):
          x_train[i] = np.transpose(ZscoreNormalization(dataa))[1000:31000]

        x_train = x_train.reshape(-1,62)

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,62)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,1,62)
        x_train_noise = x_train_noise.reshape(-1,1,62)
        
        x_test = x_train
        x_test_noise = x_train_noise

        print('x_train shape : ',x_train.shape)

        sequence_autoencoder.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test_noise, x_test))

        # build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)

        print('Encoded shape : ',encoded_x.shape)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub888888_seed2' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub' +
        #         str(subNo) + '_latentdim' + str(Config['latent_dim']) + '.mat',
        #          {'decoded_eegs': decoded_x.reshape(-1,32)})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=30000
# trialNum=4
latdim=16
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,16):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub888888_seed2'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 15):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nz_Seed3.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train), filepath, protocol=4)

In [ ]:
np.array(X_train).shape

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['decoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_decoded_dataset_64.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

# **DeNoising Seq-Sec AE IND SEED T**

In [ ]:
import math
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K
import os
import re

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer, LeakyReLU, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape, GlobalAveragePooling1D, Bidirectional
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        # at = 1- at
        output=x*at
        return output

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
def get_all_purp(root_dir):
   all_mat = dict()
   for dirName, _, fileList in os.walk(root_dir):
       for fname in fileList[:-1]:
           #print(fname)
           if '.mat' in fname:
              sub_no = int(re.split(r'/|_',fname)[-2])
              if sub_no in all_mat:
                all_exist = all_mat[sub_no]
                all_mat[sub_no] = all_exist + [dirName + '/' + fname]
              else:
                all_mat[sub_no] = [dirName + '/' + fname]
   return all_mat

In [ ]:
all_files = get_all_purp('/content/drive/MyDrive/EMOTION/Preprocessed_EEG')

In [ ]:
all_files[1]

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

batch_size = 256
original_dim = 62
epochs = 600
SNR = 1

for subNo in range(1,16): #subNo 从1到32

        timesteps = 30000  # Length of your sequences
        input_dim = 62 
        latent_dim = 16
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        # encoded = LSTM(latent_dim*2, return_sequences=True)(inputs)
        encoded = LSTM(latent_dim, return_sequences=True)(inputs)
        encoded = attention()(tf.transpose(encoded, perm = [0,2,1]))
        encoded = tf.transpose(encoded, perm = [0,2,1])

        # encoded = RepeatVector(timesteps)(encoded)
        # decoded = LSTM(latent_dim*2, return_sequences=True)(encoded)
        decoded = LSTM(input_dim, return_sequences=True)(encoded)

        # inputs = Input(shape=(timesteps, input_dim))
        # encoded = LSTM(latent_dim, activation="relu", return_sequences=False)(inputs)
        # #encoded = attention()(encoded) # this is added


        # decoded = RepeatVector(timesteps, name="bottleneck_output")(encoded)
        # decoded = LSTM(latent_dim, activation="relu", return_sequences=True)(decoded)
        # decoded = TimeDistributed(Dense(input_dim))(decoded)

        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)
        sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        
        x_train = np.zeros((15,30000,62))

        sub_data_file = []

        for filee in all_files[subNo]:
          temp = sio.loadmat(filee)
          temp = list(temp.values())[3:]
          sub_data_file = sub_data_file + temp
          break

        print('Sub Data File : ',len(sub_data_file))

        
        for i, dataa in enumerate(sub_data_file):
          x_train[i] = np.transpose(ZscoreNormalization(dataa))[1000:31000]

        x_train = x_train.reshape(-1,62)

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,62)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,30000,62)
        x_train_noise = x_train_noise.reshape(-1,30000,62)
        
        x_test = x_train
        x_test_noise = x_train_noise

        print('x_train shape : ',x_train.shape)

        sequence_autoencoder.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test_noise, x_test))

        # build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)

        print('Encoded shape : ',encoded_x.shape)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub888888_seed2_attn' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub' +
        #         str(subNo) + '_latentdim' + str(Config['latent_dim']) + '.mat',
        #          {'decoded_eegs': decoded_x.reshape(-1,32)})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=30000
# trialNum=4
latdim=16
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,16):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub888888_seed2_attn'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = trainSubData.reshape(-1,16)
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 15):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nz_Seed3_att2.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train), filepath, protocol=4)

In [ ]:
np.array(X_train).shape

In [ ]:
X_train[224].shape

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['decoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_decoded_dataset_64.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

# **DeNoising Seq-Sec AE IND Seizure**

In [ ]:
import pickle
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/x_data_ss.pkl', 'rb') as filepath:
      x_data = pickle.load(filepath)
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/y_data_ss.pkl', 'rb') as filepath:
      y_data = pickle.load(filepath)

In [ ]:
x_data = x_data.reshape(-1,18*256*20,22)
x_data.shape

In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

batch_size = 512
original_dim = 22
epochs = 25
zscore = True
SNR = 1

for subNo in range(1,12): #subNo 从1到32

        timesteps = 1  # Length of your sequences
        input_dim = 22
        latent_dim = 5
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        encoded = LSTM(latent_dim)(inputs)

        decoded = RepeatVector(timesteps)(encoded)
        decoded = LSTM(input_dim, return_sequences=True)(decoded)

        # inputs = Input(shape=(timesteps, input_dim))
        # encoded = LSTM(latent_dim, activation="relu", return_sequences=False)(inputs)
        # #encoded = attention()(encoded) # this is added


        # decoded = RepeatVector(timesteps, name="bottleneck_output")(encoded)
        # decoded = LSTM(latent_dim, activation="relu", return_sequences=True)(decoded)
        # decoded = TimeDistributed(Dense(input_dim))(decoded)

        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)
        sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            x_train = x_data[subNo-1]
            x_train = ZscoreNormalization(x_train)

        x_train = x_train[:, 0:22]

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,22)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,1,22)
        x_train_noise = x_train_noise.reshape(-1,1,22)
        
        x_test = x_train
        x_test_noise = x_train_noise

        print('x_train shape : ',x_train.shape)

        sequence_autoencoder.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test_noise, x_test))

        # build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub_sss_' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub' +
        #         str(subNo) + '_latentdim' + str(Config['latent_dim']) + '.mat',
        #          {'decoded_eegs': decoded_x.reshape(-1,32)})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=20*256
trialNum=22
latdim=5
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
X_train = []

for trainSubNo in range(1,12):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub_sss_'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 18):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse_5sss_.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train), filepath, protocol=4)

In [ ]:
np.array(X_train).shape

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['decoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_decoded_dataset_64.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/x_data_s.pkl', 'wb') as filepath:
      pickle.dump(x_data, filepath)
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/y_data_s.pkl', 'wb') as filepath:
      pickle.dump(y_data, filepath)

# **DeNoising Seq-Sec AE IND Seizure T**

In [ ]:
import pickle
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/x_data_ss_ip.pkl', 'rb') as filepath:
      x_data = pickle.load(filepath)
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/y_data_ss_ip.pkl', 'rb') as filepath:
      y_data = pickle.load(filepath)

In [ ]:
y_data.shape

In [ ]:
x_data.shape

In [ ]:
x_data = x_data.reshape(-1,18*256*20,18)
x_data.shape

In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer, LeakyReLU, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape, GlobalAveragePooling1D, Bidirectional
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return output

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

# batch_size = 512
original_dim = 18
epochs = 600
zscore = True
SNR = 1

for subNo in range(1,10): #subNo 从1到32

        timesteps = 5120  # Length of your sequences
        input_dim = 18
        latent_dim = 8
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        # encoded = LSTM(latent_dim*2, return_sequences=True)(inputs)
        encoded = LSTM(latent_dim, return_sequences=True)(inputs)
        encoded = attention()(tf.transpose(encoded, perm = [0,2,1]))
        encoded = tf.transpose(encoded, perm = [0,2,1])

        # encoded = RepeatVector(timesteps)(encoded)
        # decoded = LSTM(latent_dim*2, return_sequences=True)(encoded)
        decoded = LSTM(input_dim, return_sequences=True)(encoded)
        # inputs = Input(shape=(timesteps, input_dim))
        # encoded = LSTM(latent_dim, activation="relu", return_sequences=False)(inputs)
        # #encoded = attention()(encoded) # this is added


        # decoded = RepeatVector(timesteps, name="bottleneck_output")(encoded)
        # decoded = LSTM(latent_dim, activation="relu", return_sequences=True)(decoded)
        # decoded = TimeDistributed(Dense(input_dim))(decoded)

        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)
        sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            x_train = x_data[subNo-1]
            x_train = ZscoreNormalization(x_train)

        x_train = x_train[:, 0:18]

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,18)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,5120,18)
        x_train_noise = x_train_noise.reshape(-1,5120,18)
        
        x_test = x_train
        x_test_noise = x_train_noise

        batch_size = x_train.shape[0]

        print('x_train shape : ',x_train.shape)

        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        callbacks = [es]

        sequence_autoencoder.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test_noise, x_test),
                callbacks = callbacks)

        # build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)

        print('Encoded Shape : ',encoded_x.shape)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub_sss_attn__ip' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub' +
        #         str(subNo) + '_latentdim' + str(Config['latent_dim']) + '.mat',
        #          {'decoded_eegs': decoded_x.reshape(-1,32)})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=20*256
trialNum=22
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
X_train = []

for trainSubNo in range(1,10):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub_sss_attn__ip'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = trainSubData.reshape(-1,8)
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 18):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse_5sss_attnn__ip.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train), filepath, protocol=4)

In [ ]:
np.array(X_train).shape

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['decoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_decoded_dataset_64.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/x_data_s.pkl', 'wb') as filepath:
      pickle.dump(x_data, filepath)
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/y_data_s.pkl', 'wb') as filepath:
      pickle.dump(y_data, filepath)

# **DeNoising Seq-Sec AE with ATTn IND**

In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        #at = 1 - at
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
batch_size = 512
original_dim = 32
epochs = 20
zscore = True
SNR = 1

for subNo in range(1,33): #subNo 从1到32

        timesteps = 1  # Length of your sequences
        input_dim = 32 
        latent_dim = 8
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        encoded = LSTM(latent_dim, activation="relu", return_sequences=True)(inputs)
        #encoded = Dense(latent_dim, activation='relu')(inputs)
        encoded = attention()(encoded) # this is added
        #encoded = Dense(latent_dim, activation='relu')(encoded)

        #decoded = Dense(input_dim)(encoded)
        decoded = RepeatVector(timesteps, name="bottleneck_output")(encoded)
        decoded = LSTM(input_dim, activation="sigmoid", return_sequences=True)(decoded)
        
        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)
        sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']

        x_train = np.transpose(x_train)[:, 0:32]

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,32)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,1,32)
        x_train_noise = x_train_noise.reshape(-1,1,32)
        
        x_test = x_train
        x_test_noise = x_train_noise

        print('x_train shape : ',x_train.shape)

        sequence_autoencoder.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test_noise, x_test))

        # build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub77' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/VAE/encoded_eegs_2vae_z_sub/encoded_eegs_2vae_z_sub' +
        #             str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #             {'encoded_eegs_z': encoded_x_z})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        #sio.savemat('D:\\VAE Experiment\\DEAP\\decoded_eegs_2vae\\decoded_eegs_2vae_sub' +
        #            str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #            {'decoded_eegs': decoded_x})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub77'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAEA_ecoded_dataset_8mse.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape


# **Denoising Seq to Seq Model Combined**

In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

batch_size = 512
original_dim = 32
zscore = True

X_train = []

for subNo in tqdm(range(1,33)): #subNo 从1到32
        #### train the VAE on normalized (z-score) multi-channel EEG data
        print('subNo: '+str(subNo))
        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        print('shape : ',x_train.shape)
        X_train.append(np.transpose(x_train)[:, 0:32])




In [ ]:
np.array(X_train).shape

In [ ]:
SNR = 1000

x_train = np.array(X_train)

noise = get_white_noise(x_train.reshape(-1),SNR).reshape(32,-1,32)

x_train_noise = np.add(x_train, noise)

x_train = x_train.reshape(-1,1,32)
x_train_noise = x_train_noise.reshape(-1,1,32)

x_test = x_train
x_test_noise = x_train_noise

print('x_train shape : ',x_train.shape)

In [ ]:
epochs = 30

timesteps = 1  # Length of your sequences
input_dim = 32 
latent_dim = 16
lam = 1e-4

inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

sequence_autoencoder.fit(x_train_noise, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test_noise, x_test))

In [ ]:
x_train_for_decode = x_train.reshape(32,-1,1,32)
for subNo, x_train_ind in tqdm(enumerate(x_train_for_decode)):
        # build a model to project inputs
        encoded_x = encoder.predict(x_train_ind)
        

        print('encoded_x shape : ',encoded_x.shape)
        
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub' +
                    str(subNo+1) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        
        decoded_x = sequence_autoencoder.predict(x_train_ind)

        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train_ind.reshape(-1)))

In [ ]:
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=16
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/encoded_eegs_1DSEQae/encoded_eegs_1DSEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_all_encoded_dataset_16.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

# **Denoising Seq-Seq VAE IND**

In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
'''
 Implementation of a seq-2-seq Variational Autoendocer
 Inspired by Neural Machine Translation
 #Reference
 - Auto-Encoding Variational Bayes
   https://arxiv.org/abs/1312.6114
'''

from datetime import datetime

import matplotlib.pyplot as plt


import numpy as np
from scipy.stats import norm
from scipy.signal import square

from sklearn.model_selection import train_test_split
from keras.layers import (Input, Dense, Lambda, Layer, LSTM, Dropout,
			  TimeDistributed, Embedding, RepeatVector)
from keras.callbacks import TensorBoard, Callback
from keras.optimizers import RMSprop, Adagrad, Adam
from keras.models import load_model, Model
from keras import backend as K
from keras import metrics
from keras.losses import mse, binary_crossentropy, mean_squared_error


import numpy as np
import pickle

from tqdm import tqdm
import numpy as np
import scipy.io as sio
import h5py

import tensorflow as tf
tf.compat.v1.disable_eager_execution()


Config = {
    'seq_length': 1,
    'feature_size': 7680,
    'batch_size': 40,
    'latent_dim': 1920,
    'epochs': 10,
    'optimizer': Adam(lr=0.001),

    # model architecture
    'dense_1': 128,
    'dense_2': 64,
    'feature_rep_size': 128,  # sequence abstract feature representation
}

epsilon_std = 1.0


main_input = None  # input tensor placeholder


def log_dir():
    """
    Inner function of create_model()
    Creates unique datetime string for training dir location
    """
    date = datetime.now()
    r = '{0}_{1:02d}_{2:02d}_'.format(date.year, date.month, date.day)
    r += '{0:02d}_{1:02d}_{2:02d}'.format(date.hour, date.minute, date.second)
    return r


def roll(x):
    shifted = np.zeros_like(x)
    shifted[:, :-1, :] = x[0:, 1:, 0:]
    return shifted


# def SequenceEncoder():
#     """
#     LSTM encoder with the last layer prediction as the input to the dense layer which provides
#     variational parameters `guassian mean` and `guassian variance`
#     """

#     global main_input


#     main_input = Input(shape=(1, Config['feature_size']))  # LSTM
#     seq_1, state_h, state_c = LSTM(Config['feature_rep_size'], return_state=True)(main_input)

#     z_mean, z_log_var = (Dense(Config['latent_dim'])(state_h),
#                          Dense(Config['latent_dim'])(state_h))
#     return z_mean, z_log_var, main_input


# def SequenceDecoder(z, generator=False):
#     """
#     LSTM decoder conditioned on the output of the dense layer symetrical to the encoder
#     """

#     hidden_1 = Dense(Config['dense_1'], activation='elu')(z)
#     state_1 = Dense(Config['feature_size'])(hidden_1)
#     state_2 = Dense(Config['feature_size'])(hidden_1)
#     lstm = LSTM(Config['feature_size'], return_sequences=True, activation='sigmoid')(main_input, [state_1, state_2])

#     return lstm


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], Config['latent_dim']), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon


# # Custom loss layer
# class VariationalLayer(Layer):
#     """
#     `Bottle Neck Layer`. This layer provides the kl-divergence loss function
#     """

#     def __init__(self, **kwargs):
#         self.is_placeholder = True
#         self.iter = 0
#         self.target = Input(shape=(None, Config['feature_size']))  # original time sequence shift once
#         super(VariationalLayer, self).__init__(**kwargs)
        

#     def call(self, x, x_decoded_mean):
#         """
#         kl-divergence loss plus reconstrucion loss 
#         """

#         self.xent_loss = K.mean(metrics.mean_squared_error(self.target, x_decoded_mean), axis=1)  # sum error over time

#         self.kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
#         return K.mean(self.xent_loss + self.kl_loss)

#     # def call(self, inputs):
#     #     x = inputs[0]
#     #     x_decoded_mean = inputs[1]
#     #     loss = self.vae_loss(x, x_decoded_mean)
#     #     self.add_loss(loss, inputs=inputs)
#     #     # We won't actually use the output.
#     #     return x

    

class Anealing(Callback):
    """
    scaling the kl-divergence loss by anealing it with a smooth function: tanh in this instance.
    the tanh in this case depends on the number of epochs for the the rate of growth.
    """

    def tanh_aneal(self, iter):
        return (K.tanh((iter - 5.) / 0.25) + 1.) / 2.

    def on_batch_end(self, epoch, logs={}):
        return
        iter_num = K.tf.cast(self.model.optimizer.iterations, K.tf.float32)
        aneal_factor = K.eval(self.tanh_aneal(iter_num))
        self.model.get_layer("loss_layer").xent_loss *= aneal_factor




In [ ]:
SNR = 1
zscore = True



for subNo in range(1,33):  

      print('subNo: '+str(subNo))

      # if zscore:
      #     sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
      #     x_train = sub_data_file['zscore_data']
      # else:
      #     sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
      #     x_train = sub_data_file['minmax_data']



      x_train = np.transpose(x_train)[:, 0:32]

      noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,32)
      #print(noise)
      x_train_noise = np.add(x_train, noise)
      x_train = x_train.reshape(-1,1,32)
      x_train_noise = x_train_noise.reshape(-1,1,32)

      x_test = x_train
      x_test_noise = x_train_noise

      print('x_train shape : ',x_train.shape)
      
      main_input = Input(shape=(1, Config['feature_size']))  # LSTM
      seq_1, state_h, state_c = LSTM(Config['feature_rep_size'], return_state=True)(main_input)

      z_mean, z_log_var = (Dense(Config['latent_dim'])(state_h),
                          Dense(Config['latent_dim'])(state_h))
      # return z_mean, z_log_var, main_input


      # note that "output_shape" isn't necessary with the TensorFlow backend
      z = Lambda(sampling, output_shape=(Config['latent_dim'],))([z_mean, z_log_var])

      # we instantiate these layers separately so as to reuse them later
      hidden_1 = Dense(Config['dense_1'], activation='elu')(z)
      state_1 = Dense(Config['feature_size'])(hidden_1)
      state_2 = Dense(Config['feature_size'])(hidden_1)
      lstm = LSTM(Config['feature_size'], return_sequences=True, activation='sigmoid')(main_input, [state_1, state_2])


      # build model graph
      # vae_layer = VariationalLayer(name='loss_layer')
      #y = vae_layer([x, x_decoded_mean])

      # xent_loss = K.mean(binary_crossentropy(x, x_decoded_mean), axis=1)  # sum error over time

      # kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
      # vae_loss =  K.mean(xent_loss + kl_loss)

      def vae_loss(inputs, outputs):
        xent_loss = K.mean(mean_squared_error(inputs, outputs), axis=1)  # sum error over time
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        vae_loss =  K.mean(xent_loss + kl_loss)
        return vae_loss


      # compile VAE model
      vae = Model(main_input, lstm)
      vae.compile(optimizer='rmsprop', loss=vae_loss)


      # build encoder and decoder graph nodes
      #encoder_layer = Model(main_input, z, name='encoder')

      encoder = Model(main_input, z)
      decoder = Model(main_input, lstm)

      #print(vae.summary())
      print(x_train_noise.shape)
      print("[ INFO ] Ready to train model")



      #shifted_dataset = roll(dataset)
      vae.fit(x_train_noise, x_train,
            shuffle=True,
            epochs=Config['epochs'],
            #validation_split=0.1,
            batch_size=Config['batch_size'],
            validation_data=(x_test_noise, x_test))
            #callbacks=[TensorBoard(log_dir='/tmp/dumps/seq_model_' + log_dir()),
            #            Anealing()])
      # build a model to project inputs
      decoded_x = vae.predict(x_train)
      encoded_x = encoder.predict(x_train)

      print('encoded_x shape : ',encoded_x.shape)
      print('decoded_x shape : ',decoded_x.shape)

      sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQVAE/encoded_eegs_1DSEQvae/encoded_eegs_1DSEQvae_sub__11' +
                str(subNo) + '_latentdim' + str(Config['latent_dim']) + '.mat',
                {'encoded_eegs': encoded_x})
      

      print('Mean Difference : ', np.mean(abs(decoded_x.reshape(-1))-abs(x_train.reshape(-1))))

      sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQVAE/decoded_eegs_1DSEQvae/decoded_eegs_1DSEQvae_sub' +
                str(subNo) + '_latentdim' + str(Config['latent_dim']) + '.mat',
                 {'decoded_eegs': decoded_x.reshape(-1,32)})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQVAE/encoded_eegs_1DSEQvae/encoded_eegs_1DSEQvae_sub__11_time'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQVAE_ecoded_dataset_8_.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQVAE/decoded_eegs_1DSEQvae/decoded_eegs_1DSEQvae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['decoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQVAE_decoded_dataset_64.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(10) + '.mat', 'r')
x_train = np.array(sub_data_file['zscore_data']).transpose()[:,:32]

In [ ]:
x_train.shape

In [ ]:
np.array(X_train).shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.array(X_train)[0,500:600,14], color = 'red')
plt.plot(x_train[500:600,14])

# **VQVAE-MLP**

In [ ]:
from __future__ import print_function
import abc

import numpy as np
import logging
import torch
import torch.utils.data
from torch import nn
from torch.nn import functional as F

  
import numpy as np
import torch
from torch import nn
from torch.autograd import Function, Variable
import torch.nn.functional as F

from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer

class NearestEmbedFunc(Function):
    """
    Input:
    ------
    x - (batch_size, emb_dim, *)
        Last dimensions may be arbitrary
    emb - (emb_dim, num_emb)
    """
    @staticmethod
    def forward(ctx, input, emb):
        if input.size(1) != emb.size(0):
            raise RuntimeError('invalid argument: input.size(1) ({}) must be equal to emb.size(0) ({})'.
                               format(input.size(1), emb.size(0)))

        # save sizes for backward
        ctx.batch_size = input.size(0)
        ctx.num_latents = int(np.prod(np.array(input.size()[2:])))
        ctx.emb_dim = emb.size(0)
        ctx.num_emb = emb.size(1)
        ctx.input_type = type(input)
        ctx.dims = list(range(len(input.size())))

        # expand to be broadcast-able
        x_expanded = input.unsqueeze(-1)
        num_arbitrary_dims = len(ctx.dims) - 2
        if num_arbitrary_dims:
            emb_expanded = emb.view(emb.shape[0], *([1] * num_arbitrary_dims), emb.shape[1])
        else:
            emb_expanded = emb

        # find nearest neighbors
        dist = torch.norm(x_expanded - emb_expanded, 2, 1)
        _, argmin = dist.min(-1)
        shifted_shape = [input.shape[0], *list(input.shape[2:]) ,input.shape[1]]
        result = emb.t().index_select(0, argmin.view(-1)).view(shifted_shape).permute(0, ctx.dims[-1], *ctx.dims[1:-1])

        ctx.save_for_backward(argmin)
        return result.contiguous(), argmin

    @staticmethod
    def backward(ctx, grad_output, argmin=None):
        grad_input = grad_emb = None
        if ctx.needs_input_grad[0]:
            grad_input = grad_output

        if ctx.needs_input_grad[1]:
            argmin, = ctx.saved_variables
            latent_indices = torch.arange(ctx.num_emb).type_as(argmin)
            idx_choices = (argmin.view(-1, 1) == latent_indices.view(1, -1)).type_as(grad_output.data)
            n_idx_choice = idx_choices.sum(0)
            n_idx_choice[n_idx_choice == 0] = 1
            idx_avg_choices = idx_choices / n_idx_choice
            grad_output = grad_output.permute(0, *ctx.dims[2:], 1).contiguous()
            grad_output = grad_output.view(ctx.batch_size * ctx.num_latents, ctx.emb_dim)
            grad_emb = torch.sum(grad_output.data.view(-1, ctx.emb_dim, 1) *
                                 idx_avg_choices.view(-1, 1, ctx.num_emb), 0)
        return grad_input, grad_emb, None, None


def nearest_embed(x, emb):
    return NearestEmbedFunc().apply(x, emb)


class NearestEmbed(nn.Module):
    def __init__(self, num_embeddings, embeddings_dim):
        super(NearestEmbed, self).__init__()
        self.weight = nn.Parameter(torch.rand(embeddings_dim, num_embeddings))

    def forward(self, x, weight_sg=False):
        """Input:
        ---------
        x - (batch_size, emb_size, *)
        """
        return nearest_embed(x, self.weight.detach() if weight_sg else self.weight)


# adapted from https://github.com/rosinality/vq-vae-2-pytorch/blob/master/vqvae.py#L25
# that adapted from https://github.com/deepmind/sonnet


class NearestEmbedEMA(nn.Module):
    def __init__(self, n_emb, emb_dim, decay=0.99, eps=1e-5):
        super(NearestEmbedEMA, self).__init__()
        self.decay = decay
        self.eps = eps
        self.embeddings_dim = emb_dim
        self.n_emb = n_emb
        self.emb_dim = emb_dim
        embed = torch.rand(emb_dim, n_emb)
        self.register_buffer('weight', embed)
        self.register_buffer('cluster_size', torch.zeros(n_emb))
        self.register_buffer('embed_avg', embed.clone())

    def forward(self, x):
        """Input:
        ---------
        x - (batch_size, emb_size, *)
        """

        dims = list(range(len(x.size())))
        x_expanded = x.unsqueeze(-1)
        num_arbitrary_dims = len(dims) - 2
        if num_arbitrary_dims:
            emb_expanded = self.weight.view(self.emb_dim, *([1] * num_arbitrary_dims), self.n_emb)
        else:
            emb_expanded = self.weight

        # find nearest neighbors
        dist = torch.norm(x_expanded - emb_expanded, 2, 1)
        _, argmin = dist.min(-1)
        shifted_shape = [x.shape[0], *list(x.shape[2:]), x.shape[1]]
        result = self.weight.t().index_select(0, argmin.view(-1)).view(shifted_shape).permute(0, dims[-1], *dims[1:-1])

        if self.training:
            latent_indices = torch.arange(self.n_emb).type_as(argmin)
            emb_onehot = (argmin.view(-1, 1) == latent_indices.view(1, -1)).type_as(x.data)
            n_idx_choice = emb_onehot.sum(0)
            n_idx_choice[n_idx_choice == 0] = 1
            flatten = x.permute(1, 0, *dims[-2:]).contiguous().view(x.shape[1], -1)

            self.cluster_size.data.mul_(self.decay).add_(
                1 - self.decay, n_idx_choice
            )
            embed_sum = flatten @ emb_onehot
            self.embed_avg.data.mul_(self.decay).add_(1 - self.decay, embed_sum)

            n = self.cluster_size.sum()
            cluster_size = (
                (self.cluster_size + self.eps) / (n + self.n_emb * self.eps) * n
            )
            embed_normalized = self.embed_avg / cluster_size.unsqueeze(0)
            self.weight.data.copy_(embed_normalized)

        return result, argmin

In [ ]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as tf

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def init_hidden(x: torch.Tensor, hidden_size: int, num_dir: int = 1, xavier: bool = True):
    """
    Initialize hidden.
    Args:
        x: (torch.Tensor): input tensor
        hidden_size: (int):
        num_dir: (int): number of directions in LSTM
        xavier: (bool): wether or not use xavier initialization
    """
    if xavier:
        return nn.init.xavier_normal_(torch.zeros(num_dir, x.size(0), hidden_size)).to(device)
    return Variable(torch.zeros(num_dir, x.size(0), hidden_size)).to(device)

class VQ_VAE(nn.Module):
    """Vector Quantized AutoEncoder for mnist"""
    def __init__(self, hidden=8, k=64, vq_coef=0.2, comit_coef=0.4, **kwargs):
        super(VQ_VAE, self).__init__()

        self.emb_size = k
        # self.fc1 = nn.Linear(32, 16)
        # self.fc2 = nn.Linear(16, hidden)
        # self.lstm = nn.LSTM(input_size=32, hidden_size=hidden)
        # self.fc3 = nn.Linear(hidden, 16)
        # self.fc4 = nn.Linear(16, 32)
        self.lstm_d = nn.LSTM(input_size=hidden, hidden_size=32, bidirectional=False)
        self.fc_d = nn.Linear(32, 32)

        self.emb = NearestEmbed(k, hidden)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.vq_coef = vq_coef
        self.comit_coef = comit_coef
        self.hidden = hidden
        self.input_size = 32
        self.seq_len = 1
        self.out_feats = 32
        self.encoder_hidden_size = 8
        self.decoder_hidden_size = 8
        self.mse = 0
        self.vq_loss = torch.zeros(1)
        self.commit_loss = 0

        self.lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden,
            num_layers=1
        )
        self.attn = nn.Linear(
            in_features=2 * self.hidden + self.seq_len,
            out_features=1
        )
        self.softmax = nn.Softmax(dim=1)

        


    def encode(self, x):
        # h1 = self.relu(self.fc1(x))
        # h2 = self.fc2(h1)

        # return h2.view(-1, self.hidden)

        x = x.view(-1,1,32)
        
        h_t, c_t = (init_hidden(x, self.hidden),
                    init_hidden(x, self.hidden))
        
        attentions, input_encoded = (Variable(torch.zeros(x.size(0), self.seq_len, self.input_size)),
                                     Variable(torch.zeros(x.size(0), self.seq_len, self.hidden)))
      
        for t in range(1):
            xx = torch.cat((h_t.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           c_t.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           x.permute(0, 2, 1).to(device)), dim=2).to(
                device)  # bs * input_size * (2 * hidden_dim + seq_len)

            e_t = self.attn(xx.view(-1, self.hidden * 2 + self.seq_len))  # (bs * input_size) * 1
            a_t = self.softmax(e_t.view(-1, self.input_size)).to(device)  # (bs, input_size)

            weighted_input = torch.mul(a_t, x[:, t, :].to(device))  # (bs * input_size)

            self.lstm.flatten_parameters()

            # print('error : ',self.lstm(x[:, 0, :].unsqueeze(0), (h_t, c_t)))
            (h_t, c_t) = self.lstm(weighted_input.unsqueeze(0), (h_t, c_t))
            input_encoded[:, t, :] = h_t
          
        return input_encoded.view(-1, self.hidden).to(device)

        

    def decode(self, z):
        # h3 = self.relu(self.fc3(z))
        # return torch.tanh(self.fc4(h3))

        z = z.view(-1,1,self.hidden)

        h_t, c_t = (init_hidden(z, 32),
                    init_hidden(z, 32))
        

        for t in range(1):
            inp = z[:, t].unsqueeze(0)#.unsqueeze(2)
            # print('error : ',inp.shape)
            lstm_out, (h_t, c_t) = self.lstm_d(inp, (h_t, c_t))
        return lstm_out.view(-1, 1, 32)
        # # return self.fc_d(lstm_out.squeeze(0)).view(-1, 1, 32)
        # return return self.fc_d(lstm_out.view(-1, 32)).view(-1, 1, 32)

    def forward(self, x, eval = False):
        z_e = self.encode(x.view(-1, 32))
        # print('error : ',self.emb(z_e.detach()))
        z_q = self.emb(z_e, weight_sg=True)[0].view(-1, self.hidden)
        emb = self.emb(z_e.detach())[0].view(-1, self.hidden)
        if not(eval):
          return self.decode(z_q), z_e, emb
        else:
          return z_q, self.decode(z_q)

    def sample(self, size):
        sample = torch.randn(size, self.emb_size, int(self.hidden / self.emb_size))
        if self.cuda():
            sample = sample.cuda()
        emb, _ = self.emb(sample)
        sample = self.decode(emb(sample).view(-1, self.hidden)).cpu()
        return sample

    def loss_function(self, x, recon_x, z_e, emb):
        self.mse = F.mse_loss(recon_x, x)

        self.vq_loss = torch.mean(torch.norm((emb - z_e.detach())**2, 2, 1))
        self.commit_loss = torch.mean(torch.norm((emb.detach() - z_e)**2, 2, 1))

        return self.mse + self.vq_coef*self.vq_loss + self.comit_coef*self.commit_loss

    def latest_losses(self):
        return {'reconst': self.mse, 'vq': self.vq_loss, 'commitment': self.commit_loss}

In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
def train(epoch, model, train_loader, optimizer, cuda, log_interval):#, save_path, args, writer):
    model.train()
    loss_dict = model.latest_losses()
    losses = {k + '_train': 0 for k, v in loss_dict.items()}
    epoch_losses = {k + '_train': 0 for k, v in loss_dict.items()}
    start_time = time.time()
    batch_idx, data = None, None
    for batch_idx, data in enumerate(train_loader):
        if cuda:
            data = torch.Tensor(data).to('cuda', non_blocking=True)
        optimizer.zero_grad()
        outputs = model(data)
        loss = model.loss_function(data, *outputs)
        #print(loss.items())
        loss.backward()
        optimizer.step()
        latest_losses = model.latest_losses()
        for key in latest_losses:
            losses[key + '_train'] += float(latest_losses[key])
            epoch_losses[key + '_train'] += float(latest_losses[key])

        if batch_idx % log_interval == 0:
            for key in latest_losses:
                losses[key + '_train'] /= log_interval
            loss_string = ' '.join(['{}: {:.6f}'.format(k, v) for k, v in losses.items()])
            print('Train Epoch: {epoch} [{batch:5d}/{total_batch} ({percent:2d}%)]   time:'
                         ' {time:3.2f}   {loss}'
                         .format(epoch=epoch, batch=batch_idx * len(data), total_batch=len(train_loader) * len(data),
                                 percent=int(100. * batch_idx / len(train_loader)),
                                 time=time.time() - start_time,
                                 loss=loss_string))
            start_time = time.time()
            # logging.info('z_e norm: {:.2f}'.format(float(torch.mean(torch.norm(outputs[1][0].contiguous().view(256,-1),2,0)))))
            # logging.info('z_q norm: {:.2f}'.format(float(torch.mean(torch.norm(outputs[2][0].contiguous().view(256,-1),2,0)))))
            for key in latest_losses:
                losses[key + '_train'] = 0
        # if batch_idx == (len(train_loader) - 1):
        #     save_reconstructed_images(data, epoch, outputs[0], save_path, 'reconstruction_train')

        #     write_images(data, outputs, writer, 'train')

        # if args.dataset in ['imagenet', 'custom'] and batch_idx * len(data) > args.max_epoch_samples:
        #     break

    for key in epoch_losses:
        # if args.dataset != 'imagenet':
        #     epoch_losses[key] /= (len(train_loader.dataset) / train_loader.batch_size)
        # else:
        epoch_losses[key] /= (len(train_loader) / 128)
    # loss_string = '\t'.join(['{}: {:.6f}'.format(k, v) for k, v in epoch_losses.items()])
    # logging.info('====> Epoch: {} {}'.format(epoch, loss_string))
    # writer.add_histogram('dict frequency', outputs[3], bins=range(args.k + 1))
    # model.print_atom_hist(outputs[3])
    return epoch_losses

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector, TimeDistributed, Layer
from keras.models import Model, Sequential
import scipy.io as sio
import h5py
# import tensorflow as tf
import keras.backend as K

import os
import sys
import time
import logging
import argparse

import torch.utils.data
from torch import optim
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
from torch.nn import functional as tf


batch_size = 512
original_dim = 32
epochs = 15
zscore = True
SNR = 1

for subNo in range(1,33): #subNo 从1到32

        timesteps = 1  # Length of your sequences
        input_dim = 32 
        latent_dim = 8
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']

        x_train = np.transpose(x_train)[:, 0:32]

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,32)

        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,128,1,32)
        x_train_noise = x_train_noise.reshape(-1,32)
        
        x_test = x_train
        x_test_noise = x_train_noise

        print('x_train shape : ',x_train.shape)

        model = VQ_VAE(8, k=4).cuda()
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        scheduler = optim.lr_scheduler.StepLR(optimizer, 20, 0.5,)

        for epoch in range(epochs):
          losses = train(epoch, model, x_train, optimizer, cuda = True, log_interval = 256)
          print('losses : ',losses)

        model.eval()
        if True:
            data = torch.Tensor(x_train).to('cuda', non_blocking=True)
        encoded_x, decoded_x = model(data, True)

        print('encoded_x shape : ',encoded_x.shape)
        print('decoded_x shape : ',decoded_x.shape)

        encoded_x = encoded_x.cpu().detach().numpy()
        decoded_x = decoded_x.cpu().detach().numpy()

        # build a model to project inputs
        # decoded_x = sequence_autoencoder.predict(x_train)
        # encoded_x = encoder.predict(x_train)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/VQVAE/encoded_eegs_1VQVAEae/encoded_eegs_1VQVAEae_sub64l' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/DSEQAE/decoded_eegs_1DSEQae/decoded_eegs_1DSEQae_sub' +
        #         str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #          {'decoded_eegs': decoded_x.reshape(-1,32)})

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/VQVAE/encoded_eegs_1VQVAEae/encoded_eegs_1VQVAEae_sub64l'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/VQVAE_ecoded_dataset_8_64l.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

# **Seq-Seq LSTM AE Combined**

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

batch_size = 512
original_dim = 32
zscore = True

X_train = []

for subNo in tqdm(range(1,33)): #subNo 从1到32
        #### train the VAE on normalized (z-score) multi-channel EEG data
        print('subNo: '+str(subNo))
        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        print('shape : ',x_train.shape)
        X_train.append(np.transpose(x_train)[:, 0:32])




In [ ]:
np.array(X_train).shape

In [ ]:
x_train = np.array(X_train).reshape(-1,1,32)
x_test = x_train

print('x_train shape : ',x_train.shape)

In [ ]:
epochs = 5

timesteps = 1  # Length of your sequences
input_dim = 32 
latent_dim = 32

inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

sequence_autoencoder.fit(x_train, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test))

In [ ]:
x_train_for_decode = x_train.reshape(32,-1,1,32)
for subNo, x_train_ind in tqdm(enumerate(x_train_for_decode)):
        # build a model to project inputs
        encoded_x = encoder.predict(x_train_ind)
        

        print('encoded_x shape : ',encoded_x.shape)
        
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/SEQAE/encoded_eegs_1SEQae/encoded_eegs_1SEQae_sub' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        
        decoded_x = sequence_autoencoder.predict(x_train_ind)

        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train_ind.reshape(-1)))

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=32
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/SEQAE/encoded_eegs_1SEQae/encoded_eegs_1SEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/SEQAE_all_ecoded_dataset_32.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

# **Contractive Seq to Seq Model IND**

In [ ]:
import math

def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

batch_size = 512
original_dim = 32
epochs = 30
zscore = True
SNR = 1000

for subNo in range(1,33): #subNo 从1到32

        timesteps = 1  # Length of your sequences
        input_dim = 32 
        latent_dim = 8
        lam = 1e-4
        print('subNo: '+str(subNo)+' latend_dim: '+str(latent_dim))

        inputs = Input(shape=(timesteps, input_dim))
        #print(inputs.shape)
        encoded = LSTM(latent_dim)(inputs)
        encoded = Dense(latent_dim, activation='sigmoid', name='encoded')(encoded)
        #print(encoded.shape)
        # decoded = Dense(input_dim)(encoded)
        decoded = RepeatVector(timesteps)(encoded)
        #print(decoded.shape)
        decoded = LSTM(input_dim, return_sequences=True)(decoded)
        #print(decoded.shape)
        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)


        mse = K.mean(K.square(inputs - decoded), axis=2)
        W = K.variable(value=sequence_autoencoder.get_layer('encoded').get_weights()[0])  # N x N_hidden
        W = K.transpose(W)  # N_hidden x N
        h = sequence_autoencoder.get_layer('encoded').output

        dh = h * (1 - h)  # N_batch x N_hidden

        # N_batch x N_hidden * N_hidden x 1 = N_batch x 1
        contractive = lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)

        ae_loss = mse + contractive
        sequence_autoencoder.add_loss(ae_loss)

        sequence_autoencoder.compile(optimizer='adam')

        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        x_train = np.transpose(x_train)[:, 0:32]

        noise = get_white_noise(x_train.reshape(-1),SNR).reshape(-1,32)
        #print(noise)
        x_train_noise = np.add(x_train, noise)
        x_train = x_train.reshape(-1,1,32)
        x_train_noise = x_train_noise.reshape(-1,1,32)

        x_test = x_train
        x_test_noise = x_train_noise

        # print('x_train shape : ',x_train.shape)

        print('x_train shape : ',x_train.shape)

        sequence_autoencoder.fit(x_train_noise, x_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test_noise, x_test))

        # build a model to project inputs
        decoded_x = sequence_autoencoder.predict(x_train)
        encoded_x = encoder.predict(x_train)

        print('encoded_x : ', encoded_x.shape)
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/CSEQAE/encoded_eegs_1CSEQae/encoded_eegs_1CSEQae_sub_1d' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        # sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/VAE/encoded_eegs_2vae_z_sub/encoded_eegs_2vae_z_sub' +
        #             str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #             {'encoded_eegs_z': encoded_x_z})


        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train.reshape(-1)))
        #sio.savemat('D:\\VAE Experiment\\DEAP\\decoded_eegs_2vae\\decoded_eegs_2vae_sub' +
        #            str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
        #            {'decoded_eegs': decoded_x})

In [ ]:
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=8
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/CSEQAE/encoded_eegs_1CSEQae/encoded_eegs_1CSEQae_sub_1d'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        #trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/CSEQAE_ecoded_dataset_8__d.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

# **Contractive Seq to Seq Model Combined**

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.layers import Input, Dense, LeakyReLU, LSTM, RepeatVector
from keras.models import Model
import scipy.io as sio
import h5py
import tensorflow as tf
import keras.backend as K

batch_size = 512
original_dim = 32
zscore = True

X_train = []

for subNo in tqdm(range(1,33)): #subNo 从1到32
        #### train the VAE on normalized (z-score) multi-channel EEG data
        print('subNo: '+str(subNo))
        if zscore:
            sub_data_file = h5py.File('/content/drive/MyDrive/EMOTION/matlab_data/normalize_zscore/sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['zscore_data']
        else:
            sub_data_file = h5py.File('D:\\Processed DEAP DATA\\normalize_minmax\\sub' + str(subNo) + '.mat', 'r')
            x_train = sub_data_file['minmax_data']

        print('shape : ',x_train.shape)
        X_train.append(np.transpose(x_train)[:, 0:32])




In [ ]:
np.array(X_train).shape

In [ ]:
x_train = np.array(X_train).reshape(-1,1,32)
x_test = x_train

print('x_train shape : ',x_train.shape)

In [ ]:
epochs = 20

timesteps = 1  # Length of your sequences
input_dim = 32 
latent_dim = 32
lam = 1e-4

inputs = Input(shape=(timesteps, input_dim))
#print(inputs.shape)
encoded = LSTM(latent_dim)(inputs)
encoded = Dense(latent_dim, activation='sigmoid', name='encoded')(encoded)
#print(encoded.shape)
decoded = Dense(input_dim)(encoded)
decoded = RepeatVector(timesteps)(decoded)
#print(decoded.shape)
decoded = LSTM(input_dim, return_sequences=True)(decoded)
#print(decoded.shape)
sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)


mse = K.mean(K.square(inputs - decoded), axis=2)
W = K.variable(value=sequence_autoencoder.get_layer('encoded').get_weights()[0])  # N x N_hidden
W = K.transpose(W)  # N_hidden x N
h = sequence_autoencoder.get_layer('encoded').output

dh = h * (1 - h)  # N_batch x N_hidden

# N_batch x N_hidden * N_hidden x 1 = N_batch x 1
contractive = lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)

ae_loss = mse + contractive
sequence_autoencoder.add_loss(ae_loss)

sequence_autoencoder.compile(optimizer='adam')

sequence_autoencoder.fit(x_train, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test))

In [ ]:
x_train_for_decode = x_train.reshape(32,-1,1,32)
for subNo, x_train_ind in tqdm(enumerate(x_train_for_decode)):
        # build a model to project inputs
        encoded_x = encoder.predict(x_train_ind)
        

        print('encoded_x shape : ',encoded_x.shape)
        
     
        sio.savemat('/content/drive/MyDrive/EMOTION/matlab_data/CSEQAE/encoded_eegs_1CSEQae/encoded_eegs_1CSEQae_sub' +
                    str(subNo) + '_latentdim' + str(latent_dim) + '.mat',
                    {'encoded_eegs': encoded_x})
        
        decoded_x = sequence_autoencoder.predict(x_train_ind)

        print('Mean Difference : ', np.mean(decoded_x.reshape(-1)-x_train_ind.reshape(-1)))

In [ ]:
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=32
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/CSEQAE/encoded_eegs_1CSEQae/encoded_eegs_1CSEQae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/CSEQAE_all_encoded_dataset_32.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

# **STORE DATA SEPERATELY**

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=16
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
            l_data = pickle.load(filepath)
l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x




pred_test_f1=[]
pred_train_f1=[]
pred_test_acc=[]
pred_train_acc=[]

for testSubNo in range(1,33):
    X_test = []
    X_train = []  # list类型
    if testSubNo==1:
        Y_test = l_data[0:testSubNo * trialNum, emodim]
        Y_train = l_data[testSubNo*trialNum:32*trialNum,emodim]
    elif testSubNo==32:
        Y_test = l_data[(testSubNo-1)*trialNum:testSubNo * trialNum, emodim]
        Y_train = l_data[0:(testSubNo-1)*trialNum, emodim]
    else:
        Y_test = l_data[(testSubNo - 1) * trialNum:testSubNo * trialNum, emodim]
        Y_train1 = l_data[0:(testSubNo - 1) * trialNum, emodim]
        Y_train2 = l_data[testSubNo * trialNum:32*trialNum, emodim]
        Y_train = np.concatenate((Y_train1,Y_train2))

    print('test subNo: '+str(testSubNo))
    file1 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/AE/encoded_eegs_1ae/encoded_eegs_1ae_sub'+ str(testSubNo) + '_latentdim' + str(latdim) + '.mat')

    testSubData = file1['encoded_eegs']
    testSubData = ZscoreNormalization(testSubData)
    #print(t_data.shape)
    #构造X_test序列集
    for trialNo in range(0,40):
        trial_data = testSubData[trialNo*trialL:(trialNo+1)*trialL,:]
        #以step等间隔采样
        #trial_data = trial_data[0:trialL:step,:]
        #print(trial_data.shape)
        #将序列插入list，list中每个元素即一个序列
        X_test.append(trial_data)
    #print(len(X_test))
    #print(len(Y_test))

    # 构造X_train序列集
    for trainSubNo in range(1,33):
        if trainSubNo==testSubNo:
            continue
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/AE/encoded_eegs_1ae/encoded_eegs_1ae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
    print(len(X_train))
    print(len(Y_train))

    X_train = np.array(X_train)
    X_test = np.array(X_test)

    print('shape of X_train : ',X_train.shape)
    print('shape of X_test : ',X_test.shape)

    with open('/content/drive/MyDrive/EMOTION/encoder/AE_ecoded_dataset/final_video_X_train_60_AE' + str(testSubNo) + '.pkl', 'wb') as filepath:
              pickle.dump(X_train[:,-7680:], filepath)
    with open('/content/drive/MyDrive/EMOTION/encoder/AE_ecoded_dataset/labels_final_video_y_train_60_AE' + str(testSubNo) + '.pkl', 'wb') as filepath:
              pickle.dump(Y_train, filepath)

    with open('/content/drive/MyDrive/EMOTION/encoder/AE_ecoded_dataset/final_video_X_test_60_AE' + str(testSubNo) + '.pkl', 'wb') as filepath:
              pickle.dump(X_test[:,-7680:], filepath)
    with open('/content/drive/MyDrive/EMOTION/encoder/AE_ecoded_dataset/labels_final_video_y_test_60_AE' + str(testSubNo) + '.pkl', 'wb') as filepath:
              pickle.dump(Y_test, filepath)

# **STORE ALL DATA IN ONE**

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
import pickle

trialL=63*128
trialNum=40
latdim=32
step=128*3 #控制序列长度
method='2vae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=5

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x


X_train = []

for trainSubNo in range(1,33):
        file2 = sio.loadmat('/content/drive/MyDrive/EMOTION/matlab_data/CAE/encoded_eegs_1Cae/encoded_eegs_1Cae_sub'+ str(trainSubNo) + '_latentdim' + str(latdim) + '.mat')
        trainSubData = file2['encoded_eegs']
        trainSubData = ZscoreNormalization(trainSubData)
        for trialNo in range(0, 40):
            trial_data = trainSubData[trialNo * trialL:(trialNo + 1) * trialL, :]
            # 以step等间隔采样
            #trial_data = trial_data[0:trialL:step, :]
            # 将序列插入list，list中每个元素即一个序列
            X_train.append(trial_data)
        print(len(X_train))
print(len(X_train))

with open('/content/drive/MyDrive/EMOTION/encoder/CAE_ecoded_dataset_32.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

In [ ]:
np.array(X_train).shape

In [ ]:
with open('/content/drive/MyDrive/EMOTION/encoder/AE_ecoded_dataset_128.pkl', 'wb') as filepath:
      pickle.dump(np.array(X_train)[:,-7680:], filepath, protocol=4)

# **IND AE TRAIN**

In [ ]:
from keras.layers  import Input, LSTM, Dense, Dropout, Bidirectional
from keras.models import Model, Sequential
import h5py
import scipy.io as sio
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
import pickle

trialL=63*128
trialNum=40
latdim=16
step=128*3 #控制序列长度
method='1ae'
emodim=1 #valence 0 arousal 1
batch=40
epoch_num=10

# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#             l_data = pickle.load(filepath)
# l_data = l_data.reshape(-1,2)
# l_data[l_data<5] = 0
# l_data[l_data>=5] = 1

In [ ]:
pred_test_f1=[]
pred_train_f1=[]
pred_test_acc=[]
pred_train_acc=[]

# with open('/content/drive/MyDrive/EMOTION/encoder/AE_ecoded_dataset_32.pkl', 'rb') as filepath:
#       x_data = pickle.load(filepath)
# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#       y_data = pickle.load(filepath)

# x_data = x_data.reshape(-1,7680,32)
# y_data = y_data.reshape(-1,2)[:,0]
# y_data[y_data<5] = 0
# y_data[y_data>=5] = 1

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nz_Seed3_att2.pkl', 'rb') as filepath:
          x_data = pickle.load(filepath)
# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#           y_data = pickle.load(filepath)
# temp = sio.loadmat('/content/drive/MyDrive/EMOTION/Preprocessed_EEG/label.mat')
# temp = np.array(temp['label'])[0]
temp = np.array([ 1,  1, 0, 0,  1,  1, 0,  1,  1,  1,  1, 0,  1,  1, 0])
y_data = np.zeros((15,15))

for i in range(15):
  y_data[i] = temp

y_data = y_data.reshape(-1)

x_data = x_data.reshape(-1,30000,16)

for testSubNo in range(1,16):
    

    indd = testSubNo*15

    X_train1, X_train2, X_test, y_train1, y_train2, Y_test = x_data[:indd-15], x_data[indd:], x_data[indd-15:indd], y_data[:indd-15], y_data[indd:], y_data[indd-15:indd]
    Y_train = np.concatenate((y_train1,y_train2))
    X_train = np.concatenate((X_train1,X_train2))

    print('test subNo: '+str(testSubNo))

    X_train = np.array(X_train)
    X_test = np.array(X_test)

    print('shape of X_train : ',X_train.shape)
    print('shape of X_test : ',X_test.shape)

    #构造RNN模型
    seqL = X_train.shape[1]
    print('seqence length: '+str(seqL))
    model = Sequential()
    model.add((LSTM(200, input_shape=(seqL, latdim))))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='sigmoid'))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epoch_num, batch_size=batch)
    # predict on the training data after training...
    # Y_test_predict = model.predict_classes([X_test])
    # Y_train_predict = model.predict_classes([X_train])

    # print(metrics.f1_score(Y_test, Y_test_predict))
    # print(metrics.accuracy_score(Y_test, Y_test_predict))
    # print(metrics.f1_score(Y_train, Y_train_predict))
    # print(metrics.accuracy_score(Y_train, Y_train_predict))
    # print(classification_report(np.array(Y_test), np.array(Y_test_predict)))

    # pred_test_f1.append(metrics.f1_score(Y_test, Y_test_predict))
    # pred_test_acc.append(metrics.accuracy_score(Y_test, Y_test_predict))
    # pred_train_f1.append(metrics.f1_score(Y_train, Y_train_predict))
    # pred_train_acc.append(metrics.accuracy_score(Y_train, Y_train_predict))

# **INTRO**

In [ ]:
import os
import numpy as np
import sys
import sklearn
import pickle

#import utils

In [ ]:
from builtins import print
import numpy as np
import pandas as pd
import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'
import os
import operator

#import utils



from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelEncoder

from scipy.interpolate import interp1d
from scipy.io import loadmat


def readucr(filename):
    data = np.loadtxt(filename, delimiter=',')
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y


def create_directory(directory_path):
    if os.path.exists(directory_path):
        return None
    else:
        try:
            os.makedirs(directory_path)
        except:
            # in case another machine created the path meanwhile !:(
            return None
        return directory_path


def create_path(root_dir, classifier_name, archive_name):
    output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '/'
    if os.path.exists(output_directory):
        return None
    else:
        os.makedirs(output_directory)
        return output_directory


def read_dataset(root_dir, archive_name, dataset_name):
    datasets_dict = {}
    cur_root_dir = root_dir.replace('-temp', '')

    if archive_name == 'mts_archive':
        file_name = cur_root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'
        x_train = np.load(file_name + 'x_train.npy')
        y_train = np.load(file_name + 'y_train.npy')
        x_test = np.load(file_name + 'x_test.npy')
        y_test = np.load(file_name + 'y_test.npy')

        datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                       y_test.copy())

    elif archive_name == 'UCRArchive_2018':
        root_dir_dataset = cur_root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'
        df_train = pd.read_csv(root_dir_dataset + '/' + dataset_name + '_TRAIN.tsv', sep='\t', header=None)

        df_test = pd.read_csv(root_dir_dataset + '/' + dataset_name + '_TEST.tsv', sep='\t', header=None)

        y_train = df_train.values[:, 0]
        y_test = df_test.values[:, 0]

        x_train = df_train.drop(columns=[0])
        x_test = df_test.drop(columns=[0])

        x_train.columns = range(x_train.shape[1])
        x_test.columns = range(x_test.shape[1])

        x_train = x_train.values
        x_test = x_test.values

        # znorm
        std_ = x_train.std(axis=1, keepdims=True)
        std_[std_ == 0] = 1.0
        x_train = (x_train - x_train.mean(axis=1, keepdims=True)) / std_

        std_ = x_test.std(axis=1, keepdims=True)
        std_[std_ == 0] = 1.0
        x_test = (x_test - x_test.mean(axis=1, keepdims=True)) / std_

        datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                       y_test.copy())
    else:
        file_name = cur_root_dir + '/archives/' + archive_name + '/' + dataset_name + '/' + dataset_name
        x_train, y_train = readucr(file_name + '_TRAIN')
        x_test, y_test = readucr(file_name + '_TEST')
        datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                       y_test.copy())

    return datasets_dict


def read_all_datasets(root_dir, archive_name, split_val=False):
    datasets_dict = {}
    cur_root_dir = root_dir.replace('-temp', '')
    dataset_names_to_sort = []

    if archive_name == 'mts_archive':

        for dataset_name in MTS_DATASET_NAMES:
            root_dir_dataset = cur_root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'

            x_train = np.load(root_dir_dataset + 'x_train.npy')
            y_train = np.load(root_dir_dataset + 'y_train.npy')
            x_test = np.load(root_dir_dataset + 'x_test.npy')
            y_test = np.load(root_dir_dataset + 'y_test.npy')

            datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                           y_test.copy())
    elif archive_name == 'UCRArchive_2018':
        for dataset_name in DATASET_NAMES_2018:
            root_dir_dataset = cur_root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'

            df_train = pd.read_csv(root_dir_dataset + '/' + dataset_name + '_TRAIN.tsv', sep='\t', header=None)

            df_test = pd.read_csv(root_dir_dataset + '/' + dataset_name + '_TEST.tsv', sep='\t', header=None)

            y_train = df_train.values[:, 0]
            y_test = df_test.values[:, 0]

            x_train = df_train.drop(columns=[0])
            x_test = df_test.drop(columns=[0])

            x_train.columns = range(x_train.shape[1])
            x_test.columns = range(x_test.shape[1])

            x_train = x_train.values
            x_test = x_test.values

            # znorm
            std_ = x_train.std(axis=1, keepdims=True)
            std_[std_ == 0] = 1.0
            x_train = (x_train - x_train.mean(axis=1, keepdims=True)) / std_

            std_ = x_test.std(axis=1, keepdims=True)
            std_[std_ == 0] = 1.0
            x_test = (x_test - x_test.mean(axis=1, keepdims=True)) / std_

            datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                           y_test.copy())

    else:
        for dataset_name in DATASET_NAMES:
            root_dir_dataset = cur_root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'
            file_name = root_dir_dataset + dataset_name
            x_train, y_train = readucr(file_name + '_TRAIN')
            x_test, y_test = readucr(file_name + '_TEST')

            datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                           y_test.copy())

            dataset_names_to_sort.append((dataset_name, len(x_train)))

        dataset_names_to_sort.sort(key=operator.itemgetter(1))

        for i in range(len(DATASET_NAMES)):
            DATASET_NAMES[i] = dataset_names_to_sort[i][0]

    return datasets_dict


def get_func_length(x_train, x_test, func):
    if func == min:
        func_length = np.inf
    else:
        func_length = 0

    n = x_train.shape[0]
    for i in range(n):
        func_length = func(func_length, x_train[i].shape[1])

    n = x_test.shape[0]
    for i in range(n):
        func_length = func(func_length, x_test[i].shape[1])

    return func_length


def transform_to_same_length(x, n_var, max_length):
    n = x.shape[0]

    # the new set in ucr form np array
    ucr_x = np.zeros((n, max_length, n_var), dtype=np.float64)

    # loop through each time series
    for i in range(n):
        mts = x[i]
        curr_length = mts.shape[1]
        idx = np.array(range(curr_length))
        idx_new = np.linspace(0, idx.max(), max_length)
        for j in range(n_var):
            ts = mts[j]
            # linear interpolation
            f = interp1d(idx, ts, kind='cubic')
            new_ts = f(idx_new)
            ucr_x[i, :, j] = new_ts

    return ucr_x


def transform_mts_to_ucr_format():
    mts_root_dir = '/mnt/Other/mtsdata/'
    mts_out_dir = '/mnt/nfs/casimir/archives/mts_archive/'
    for dataset_name in MTS_DATASET_NAMES:
        # print('dataset_name',dataset_name)

        out_dir = mts_out_dir + dataset_name + '/'

        # if create_directory(out_dir) is None:
        #     print('Already_done')
        #     continue

        a = loadmat(mts_root_dir + dataset_name + '/' + dataset_name + '.mat')
        a = a['mts']
        a = a[0, 0]

        dt = a.dtype.names
        dt = list(dt)

        for i in range(len(dt)):
            if dt[i] == 'train':
                x_train = a[i].reshape(max(a[i].shape))
            elif dt[i] == 'test':
                x_test = a[i].reshape(max(a[i].shape))
            elif dt[i] == 'trainlabels':
                y_train = a[i].reshape(max(a[i].shape))
            elif dt[i] == 'testlabels':
                y_test = a[i].reshape(max(a[i].shape))

        # x_train = a[1][0]
        # y_train = a[0][:,0]
        # x_test = a[3][0]
        # y_test = a[2][:,0]

        n_var = x_train[0].shape[0]

        max_length = get_func_length(x_train, x_test, func=max)
        min_length = get_func_length(x_train, x_test, func=min)

        print(dataset_name, 'max', max_length, 'min', min_length)
        print()
        # continue

        x_train = transform_to_same_length(x_train, n_var, max_length)
        x_test = transform_to_same_length(x_test, n_var, max_length)

        # save them
        np.save(out_dir + 'x_train.npy', x_train)
        np.save(out_dir + 'y_train.npy', y_train)
        np.save(out_dir + 'x_test.npy', x_test)
        np.save(out_dir + 'y_test.npy', y_test)

        print('Done')


def calculate_metrics(y_true, y_pred, duration, y_true_val=None, y_pred_val=None):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall', 'duration'])
    res['precision'] = precision_score(y_true, y_pred, average='macro')
    res['accuracy'] = accuracy_score(y_true, y_pred)

    if not y_true_val is None:
        # this is useful when transfer learning is used with cross validation
        res['accuracy_val'] = accuracy_score(y_true_val, y_pred_val)

    res['recall'] = recall_score(y_true, y_pred, average='macro')
    res['duration'] = duration
    return res


def save_test_duration(file_name, test_duration):
    res = pd.DataFrame(data=np.zeros((1, 1), dtype=np.float), index=[0],
                       columns=['test_duration'])
    res['test_duration'] = test_duration
    res.to_csv(file_name, index=False)


def generate_results_csv(output_file_name, root_dir):
    res = pd.DataFrame(data=np.zeros((0, 7), dtype=np.float), index=[],
                       columns=['classifier_name', 'archive_name', 'dataset_name',
                                'precision', 'accuracy', 'recall', 'duration'])
    for classifier_name in CLASSIFIERS:
        for archive_name in ARCHIVE_NAMES:
            datasets_dict = read_all_datasets(root_dir, archive_name)
            for it in range(ITERATIONS):
                curr_archive_name = archive_name
                if it != 0:
                    curr_archive_name = curr_archive_name + '_itr_' + str(it)
                for dataset_name in datasets_dict.keys():
                    output_dir = root_dir + '/results/' + classifier_name + '/' \
                                 + curr_archive_name + '/' + dataset_name + '/' + 'df_metrics.csv'
                    if not os.path.exists(output_dir):
                        continue
                    df_metrics = pd.read_csv(output_dir)
                    df_metrics['classifier_name'] = classifier_name
                    df_metrics['archive_name'] = archive_name
                    df_metrics['dataset_name'] = dataset_name
                    res = pd.concat((res, df_metrics), axis=0, sort=False)

    res.to_csv(root_dir + output_file_name, index=False)
    # aggreagte the accuracy for iterations on same dataset
    res = pd.DataFrame({
        'accuracy': res.groupby(
            ['classifier_name', 'archive_name', 'dataset_name'])['accuracy'].mean()
    }).reset_index()

    return res


def plot_epochs_metric(hist, file_name, metric='loss'):
    plt.figure()
    plt.plot(hist.history[metric])
    plt.plot(hist.history['val_' + metric])
    plt.title('model ' + metric)
    plt.ylabel(metric, fontsize='large')
    plt.xlabel('epoch', fontsize='large')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(file_name, bbox_inches='tight')
    plt.close()


def save_logs_t_leNet(output_directory, hist, y_pred, y_true, duration):
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv(output_directory + 'history.csv', index=False)

    df_metrics = calculate_metrics(y_true, y_pred, duration)
    df_metrics.to_csv(output_directory + 'df_metrics.csv', index=False)

    index_best_model = hist_df['loss'].idxmin()
    row_best_model = hist_df.loc[index_best_model]

    df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float), index=[0],
                                 columns=['best_model_train_loss', 'best_model_val_loss', 'best_model_train_acc',
                                          'best_model_val_acc', 'best_model_learning_rate', 'best_model_nb_epoch'])

    df_best_model['best_model_train_loss'] = row_best_model['loss']
    df_best_model['best_model_val_loss'] = row_best_model['val_loss']
    df_best_model['best_model_train_acc'] = row_best_model['acc']
    df_best_model['best_model_val_acc'] = row_best_model['val_acc']
    df_best_model['best_model_nb_epoch'] = index_best_model

    df_best_model.to_csv(output_directory + 'df_best_model.csv', index=False)

    # plot losses
    plot_epochs_metric(hist, output_directory + 'epochs_loss.png')


def save_logs(output_directory, hist, y_pred, y_true, duration, lr=True, y_true_val=None, y_pred_val=None):
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv(output_directory + 'history.csv', index=False)

    df_metrics = calculate_metrics(y_true, y_pred, duration, y_true_val, y_pred_val)
    df_metrics.to_csv(output_directory + 'df_metrics.csv', index=False)

    index_best_model = hist_df['loss'].idxmin()
    row_best_model = hist_df.loc[index_best_model]

    df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float), index=[0],
                                 columns=['best_model_train_loss', 'best_model_val_loss', 'best_model_train_acc',
                                          'best_model_val_acc', 'best_model_learning_rate', 'best_model_nb_epoch'])

    df_best_model['best_model_train_loss'] = row_best_model['loss']
    df_best_model['best_model_val_loss'] = row_best_model['val_loss']
    df_best_model['best_model_train_acc'] = row_best_model['accuracy']
    df_best_model['best_model_val_acc'] = row_best_model['val_accuracy']
    if lr == True:
        df_best_model['best_model_learning_rate'] = row_best_model['lr']
    df_best_model['best_model_nb_epoch'] = index_best_model

    df_best_model.to_csv(output_directory + 'df_best_model.csv', index=False)

    # for FCN there is no hyperparameters fine tuning - everything is static in code

    # plot losses
    plot_epochs_metric(hist, output_directory + 'epochs_loss.png')

    return df_metrics


def visualize_filter(root_dir):
    import tensorflow.keras as keras
    classifier = 'resnet'
    archive_name = 'UCRArchive_2018'
    dataset_name = 'GunPoint'
    datasets_dict = read_dataset(root_dir, archive_name, dataset_name)

    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]

    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

    model = keras.models.load_model(
        root_dir + 'results/' + classifier + '/' + archive_name + '/' + dataset_name + '/best_model.hdf5')

    # filters
    filters = model.layers[1].get_weights()[0]

    new_input_layer = model.inputs
    new_output_layer = [model.layers[1].output]

    new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)

    classes = np.unique(y_train)

    colors = [(255 / 255, 160 / 255, 14 / 255), (181 / 255, 87 / 255, 181 / 255)]
    colors_conv = [(210 / 255, 0 / 255, 0 / 255), (27 / 255, 32 / 255, 101 / 255)]

    idx = 10
    idx_filter = 1

    filter = filters[:, 0, idx_filter]

    plt.figure(1)
    plt.plot(filter + 0.5, color='gray', label='filter')
    for c in classes:
        c_x_train = x_train[np.where(y_train == c)]
        convolved_filter_1 = new_feed_forward([c_x_train])[0]

        idx_c = int(c) - 1

        plt.plot(c_x_train[idx], color=colors[idx_c], label='class' + str(idx_c) + '-raw')
        plt.plot(convolved_filter_1[idx, :, idx_filter], color=colors_conv[idx_c], label='class' + str(idx_c) + '-conv')
        plt.legend()

    plt.savefig(root_dir + 'convolution-' + dataset_name + '.pdf')

    return 1


def viz_perf_themes(root_dir, df):
    df_themes = df.copy()
    themes_index = []
    # add the themes
    for dataset_name in df.index:
        themes_index.append(utils.constants.dataset_types[dataset_name])

    themes_index = np.array(themes_index)
    themes, themes_counts = np.unique(themes_index, return_counts=True)
    df_themes.index = themes_index
    df_themes = df_themes.rank(axis=1, method='min', ascending=False)
    df_themes = df_themes.where(df_themes.values == 1)
    df_themes = df_themes.groupby(level=0).sum(axis=1)
    df_themes['#'] = themes_counts

    for classifier in CLASSIFIERS:
        df_themes[classifier] = df_themes[classifier] / df_themes['#'] * 100
    df_themes = df_themes.round(decimals=1)
    df_themes.to_csv(root_dir + 'tab-perf-theme.csv')


def viz_perf_train_size(root_dir, df):
    df_size = df.copy()
    train_sizes = []
    datasets_dict_ucr = read_all_datasets(root_dir, archive_name='UCR_TS_Archive_2015')
    datasets_dict_mts = read_all_datasets(root_dir, archive_name='mts_archive')
    datasets_dict = dict(datasets_dict_ucr, **datasets_dict_mts)

    for dataset_name in df.index:
        train_size = len(datasets_dict[dataset_name][0])
        train_sizes.append(train_size)

    train_sizes = np.array(train_sizes)
    bins = np.array([0, 100, 400, 800, 99999])
    train_size_index = np.digitize(train_sizes, bins)
    train_size_index = bins[train_size_index]

    df_size.index = train_size_index
    df_size = df_size.rank(axis=1, method='min', ascending=False)
    df_size = df_size.groupby(level=0, axis=0).mean()
    df_size = df_size.round(decimals=2)

    print(df_size.to_string())
    df_size.to_csv(root_dir + 'tab-perf-train-size.csv')


def viz_perf_classes(root_dir, df):
    df_classes = df.copy()
    class_numbers = []
    datasets_dict_ucr = read_all_datasets(root_dir, archive_name='UCR_TS_Archive_2015')
    datasets_dict_mts = read_all_datasets(root_dir, archive_name='mts_archive')
    datasets_dict = dict(datasets_dict_ucr, **datasets_dict_mts)

    for dataset_name in df.index:
        train_size = len(np.unique(datasets_dict[dataset_name][1]))
        class_numbers.append(train_size)

    class_numbers = np.array(class_numbers)
    bins = np.array([0, 3, 4, 6, 8, 13, 9999])
    class_numbers_index = np.digitize(class_numbers, bins)
    class_numbers_index = bins[class_numbers_index]

    df_classes.index = class_numbers_index
    df_classes = df_classes.rank(axis=1, method='min', ascending=False)
    df_classes = df_classes.groupby(level=0, axis=0).mean()
    df_classes = df_classes.round(decimals=2)

    print(df_classes.to_string())
    df_classes.to_csv(root_dir + 'tab-perf-classes.csv')


def viz_perf_length(root_dir, df):
    df_lengths = df.copy()
    lengths = []
    datasets_dict_ucr = read_all_datasets(root_dir, archive_name='UCR_TS_Archive_2015')
    datasets_dict_mts = read_all_datasets(root_dir, archive_name='mts_archive')
    datasets_dict = dict(datasets_dict_ucr, **datasets_dict_mts)

    for dataset_name in df.index:
        length = datasets_dict[dataset_name][0].shape[1]
        lengths.append(length)

    lengths = np.array(lengths)
    bins = np.array([0, 81, 251, 451, 700, 1001, 9999])
    lengths_index = np.digitize(lengths, bins)
    lengths_index = bins[lengths_index]

    df_lengths.index = lengths_index
    df_lengths = df_lengths.rank(axis=1, method='min', ascending=False)
    df_lengths = df_lengths.groupby(level=0, axis=0).mean()
    df_lengths = df_lengths.round(decimals=2)

    print(df_lengths.to_string())
    df_lengths.to_csv(root_dir + 'tab-perf-lengths.csv')


def viz_plot(root_dir, df):
    df_lengths = df.copy()
    lengths = []
    datasets_dict_ucr = read_all_datasets(root_dir, archive_name='UCR_TS_Archive_2015')
    datasets_dict_mts = read_all_datasets(root_dir, archive_name='mts_archive')
    datasets_dict = dict(datasets_dict_ucr, **datasets_dict_mts)

    for dataset_name in df.index:
        length = datasets_dict[dataset_name][0].shape[1]
        lengths.append(length)

    lengths_index = np.array(lengths)

    df_lengths.index = lengths_index

    plt.scatter(x=df_lengths['fcn'], y=df_lengths['resnet'])
    plt.ylim(ymin=0, ymax=1.05)
    plt.xlim(xmin=0, xmax=1.05)
    # df_lengths['fcn']
    plt.savefig(root_dir + 'plot.pdf')


def viz_for_survey_paper(root_dir, filename='results-ucr-mts.csv'):
    df = pd.read_csv(root_dir + filename, index_col=0)
    df = df.T
    df = df.round(decimals=2)

    # get table performance per themes
    # viz_perf_themes(root_dir,df)

    # get table performance per train size
    # viz_perf_train_size(root_dir,df)

    # get table performance per classes
    # viz_perf_classes(root_dir,df)

    # get table performance per length
    # viz_perf_length(root_dir,df)

    # get plot
    viz_plot(root_dir, df)


def viz_cam(root_dir):
    import tensorflow.keras as keras
    import sklearn
    classifier = 'resnet'
    archive_name = 'UCRArchive_2018'
    dataset_name = 'GunPoint'

    if dataset_name == 'Gun_Point':
        save_name = 'GunPoint'
    else:
        save_name = dataset_name
    max_length = 2000
    datasets_dict = read_dataset(root_dir, archive_name, dataset_name)

    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    y_test = datasets_dict[dataset_name][3]

    # transform to binary labels
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
    y_train_binary = enc.transform(y_train.reshape(-1, 1)).toarray()

    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

    model = keras.models.load_model(
        root_dir + 'results/' + classifier + '/' + archive_name + '/' + dataset_name + '/best_model.hdf5')

    # filters
    w_k_c = model.layers[-1].get_weights()[0]  # weights for each filter k for each class c

    # the same input
    new_input_layer = model.inputs
    # output is both the original as well as the before last layer
    new_output_layer = [model.layers[-3].output, model.layers[-1].output]

    new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)

    classes = np.unique(y_train)

    for c in classes:
        plt.figure()
        count = 0
        c_x_train = x_train[np.where(y_train == c)]
        for ts in c_x_train:
            ts = ts.reshape(1, -1, 1)
            [conv_out, predicted] = new_feed_forward([ts])
            pred_label = np.argmax(predicted)
            orig_label = np.argmax(enc.transform([[c]]))
            if pred_label == orig_label:
                cas = np.zeros(dtype=np.float, shape=(conv_out.shape[1]))
                for k, w in enumerate(w_k_c[:, orig_label]):
                    cas += w * conv_out[0, :, k]

                minimum = np.min(cas)

                cas = cas - minimum

                cas = cas / max(cas)
                cas = cas * 100

                x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
                # linear interpolation to smooth
                f = interp1d(range(ts.shape[1]), ts[0, :, 0])
                y = f(x)
                # if (y < -2.2).any():
                #     continue
                f = interp1d(range(ts.shape[1]), cas)
                cas = f(x).astype(int)
                plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=2, vmin=0, vmax=100, linewidths=0.0)
                if dataset_name == 'Gun_Point':
                    if c == 1:
                        plt.yticks([-1.0, 0.0, 1.0, 2.0])
                    else:
                        plt.yticks([-2, -1.0, 0.0, 1.0, 2.0])
                count += 1

        cbar = plt.colorbar()
        # cbar.ax.set_yticklabels([100,75,50,25,0])
        plt.savefig(root_dir + '/temp/' + classifier + '-cam-' + save_name + '-class-' + str(int(c)) + '.png',
                    bbox_inches='tight', dpi=1080)

In [ ]:
UNIVARIATE_DATASET_NAMES = ['50words', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'Car', 'CBF',
                            'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee',
                            'Computers', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 'DiatomSizeReduction',
                            'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW',
                            'Earthquakes', 'ECG200', 'ECG5000', 'ECGFiveDays', 'ElectricDevices', 'FaceAll', 'FaceFour',
                            'FacesUCR', 'FISH', 'FordA', 'FordB', 'Gun_Point', 'Ham', 'HandOutlines',
                            'Haptics', 'Herring', 'InlineSkate', 'InsectWingbeatSound', 'ItalyPowerDemand',
                            'LargeKitchenAppliances', 'Lighting2', 'Lighting7', 'MALLAT', 'Meat', 'MedicalImages',
                            'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW',
                            'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 'NonInvasiveFatalECG_Thorax2', 'OliveOil',
                            'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme', 'Plane', 'ProximalPhalanxOutlineAgeGroup',
                            'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 'RefrigerationDevices',
                            'ScreenType', 'ShapeletSim', 'ShapesAll', 'SmallKitchenAppliances', 'SonyAIBORobotSurface',
                            'SonyAIBORobotSurfaceII', 'StarLightCurves', 'Strawberry', 'SwedishLeaf', 'Symbols',
                            'synthetic_control', 'ToeSegmentation1', 'ToeSegmentation2', 'Trace', 'TwoLeadECG',
                            'Two_Patterns', 'UWaveGestureLibraryAll', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y',
                            'uWaveGestureLibrary_Z', 'wafer', 'Wine', 'WordsSynonyms', 'Worms', 'WormsTwoClass', 'yoga']

UNIVARIATE_DATASET_NAMES_2018 = ['ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ',
                                 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown',
                                 'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'Computers', 'CricketX',
                                 'CricketY', 'CricketZ', 'Crop', 'DiatomSizeReduction',
                                 'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW',
                                 'DodgerLoopDay', 'DodgerLoopGame', 'DodgerLoopWeekend', 'Earthquakes', 'ECG200',
                                 'ECG5000', 'ECGFiveDays', 'ElectricDevices', 'EOGHorizontalSignal',
                                 'EOGVerticalSignal', 'EthanolLevel', 'FaceAll', 'FaceFour', 'FacesUCR',
                                 'FiftyWords', 'Fish', 'FordA', 'FordB', 'FreezerRegularTrain',
                                 'FreezerSmallTrain', 'Fungi', 'GestureMidAirD1', 'GestureMidAirD2',
                                 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 'GunPoint',
                                 'GunPointAgeSpan', 'GunPointMaleVersusFemale', 'GunPointOldVersusYoung',
                                 'Ham', 'HandOutlines', 'Haptics', 'Herring', 'HouseTwenty', 'InlineSkate',
                                 'InsectEPGRegularTrain', 'InsectEPGSmallTrain', 'InsectWingbeatSound',
                                 'ItalyPowerDemand', 'LargeKitchenAppliances', 'Lightning2', 'Lightning7',
                                 'Mallat', 'Meat', 'MedicalImages', 'MelbournePedestrian',
                                 'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect',
                                 'MiddlePhalanxTW', 'MixedShapesRegularTrain', 'MixedShapesSmallTrain',
                                 'MoteStrain', 'NonInvasiveFetalECGThorax1', 'NonInvasiveFetalECGThorax2',
                                 'OliveOil', 'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme',
                                 'PickupGestureWiimoteZ', 'PigAirwayPressure', 'PigArtPressure', 'PigCVP',
                                 'PLAID', 'Plane', 'PowerCons', 'ProximalPhalanxOutlineAgeGroup',
                                 'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 'RefrigerationDevices',
                                 'Rock', 'ScreenType', 'SemgHandGenderCh2', 'SemgHandMovementCh2',
                                 'SemgHandSubjectCh2', 'ShakeGestureWiimoteZ', 'ShapeletSim', 'ShapesAll',
                                 'SmallKitchenAppliances', 'SmoothSubspace', 'SonyAIBORobotSurface1',
                                 'SonyAIBORobotSurface2', 'StarLightCurves', 'Strawberry', 'SwedishLeaf',
                                 'Symbols', 'SyntheticControl', 'ToeSegmentation1', 'ToeSegmentation2', 'Trace',
                                 'TwoLeadECG', 'TwoPatterns', 'UMD', 'UWaveGestureLibraryAll',
                                 'UWaveGestureLibraryX', 'UWaveGestureLibraryY', 'UWaveGestureLibraryZ',
                                 'Wafer', 'Wine', 'WordSynonyms', 'Worms', 'WormsTwoClass', 'Yoga']

MTS_DATASET_NAMES = ['ArabicDigits', 'AUSLAN', 'CharacterTrajectories', 'CMUsubject16', 'ECG',
                     'JapaneseVowels', 'KickvsPunch', 'Libras', 'NetFlow', 'UWave', 'Wafer', 'WalkvsRun']

ITERATIONS = 5  # nb of random runs for random initializations

ARCHIVE_NAMES = ['UCRArchive_2018']

dataset_names_for_archive = {'UCRArchive_2018': UNIVARIATE_DATASET_NAMES_2018}

CLASSIFIERS = ['fcn', 'mlp', 'resnet', 'tlenet', 'mcnn', 'twiesn', 'encoder', 'mcdcnn', 'cnn', 'inception']

dataset_types = {'ElectricDevices': 'DEVICE', 'FordB': 'SENSOR',
                 'FordA': 'SENSOR', 'NonInvasiveFatalECG_Thorax2': 'ECG',
                 'NonInvasiveFatalECG_Thorax1': 'ECG', 'PhalangesOutlinesCorrect': 'IMAGE',
                 'HandOutlines': 'IMAGE', 'StarLightCurves': 'SENSOR',
                 'wafer': 'SENSOR', 'Two_Patterns': 'SIMULATED',
                 'UWaveGestureLibraryAll': 'MOTION', 'uWaveGestureLibrary_Z': 'MOTION',
                 'uWaveGestureLibrary_Y': 'MOTION', 'uWaveGestureLibrary_X': 'MOTION',
                 'Strawberry': 'SPECTRO', 'ShapesAll': 'IMAGE',
                 'ProximalPhalanxOutlineCorrect': 'IMAGE', 'MiddlePhalanxOutlineCorrect': 'IMAGE',
                 'DistalPhalanxOutlineCorrect': 'IMAGE', 'FaceAll': 'IMAGE',
                 'ECG5000': 'ECG', 'SwedishLeaf': 'IMAGE', 'ChlorineConcentration': 'SIMULATED',
                 '50words': 'IMAGE', 'ProximalPhalanxTW': 'IMAGE', 'ProximalPhalanxOutlineAgeGroup': 'IMAGE',
                 'MiddlePhalanxOutlineAgeGroup': 'IMAGE', 'DistalPhalanxTW': 'IMAGE',
                 'DistalPhalanxOutlineAgeGroup': 'IMAGE', 'MiddlePhalanxTW': 'IMAGE',
                 'Cricket_Z': 'MOTION', 'Cricket_Y': 'MOTION',
                 'Cricket_X': 'MOTION', 'Adiac': 'IMAGE',
                 'MedicalImages': 'IMAGE', 'SmallKitchenAppliances': 'DEVICE',
                 'ScreenType': 'DEVICE', 'RefrigerationDevices': 'DEVICE',
                 'LargeKitchenAppliances': 'DEVICE', 'Earthquakes': 'SENSOR',
                 'yoga': 'IMAGE', 'synthetic_control': 'SIMULATED',
                 'WordsSynonyms': 'IMAGE', 'Computers': 'DEVICE',
                 'InsectWingbeatSound': 'SENSOR', 'Phoneme': 'SENSOR',
                 'OSULeaf': 'IMAGE', 'FacesUCR': 'IMAGE',
                 'WormsTwoClass': 'MOTION', 'Worms': 'MOTION',
                 'FISH': 'IMAGE', 'Haptics': 'MOTION',
                 'Epilepsy': 'HAR', 'Ham': 'SPECTRO',
                 'Plane': 'SENSOR', 'InlineSkate': 'MOTION',
                 'Trace': 'SENSOR', 'ECG200': 'ECG',
                 'Lighting7': 'SENSOR', 'ItalyPowerDemand': 'SENSOR',
                 'Herring': 'IMAGE', 'Lighting2': 'SENSOR',
                 'Car': 'SENSOR', 'Meat': 'SPECTRO',
                 'Wine': 'SPECTRO', 'MALLAT': 'SIMULATED',
                 'Gun_Point': 'MOTION', 'CinC_ECG_torso': 'ECG',
                 'ToeSegmentation1': 'MOTION', 'ToeSegmentation2': 'MOTION',
                 'ArrowHead': 'IMAGE', 'OliveOil': 'SPECTRO',
                 'Beef': 'SPECTRO', 'CBF': 'SIMULATED',
                 'Coffee': 'SPECTRO', 'SonyAIBORobotSurfaceII': 'SENSOR',
                 'Symbols': 'IMAGE', 'FaceFour': 'IMAGE',
                 'ECGFiveDays': 'ECG', 'TwoLeadECG': 'ECG',
                 'BirdChicken': 'IMAGE', 'BeetleFly': 'IMAGE',
                 'ShapeletSim': 'SIMULATED', 'MoteStrain': 'SENSOR',
                 'SonyAIBORobotSurface': 'SENSOR', 'DiatomSizeReduction': 'IMAGE'}

themes_colors = {'IMAGE': 'red', 'SENSOR': 'blue', 'ECG': 'green',
                 'SIMULATED': 'yellow', 'SPECTRO': 'orange',
                 'MOTION': 'purple', 'DEVICE': 'gray'}

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time

class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, batch_size=64, lr=0.001,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.verbose = verbose

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > self.bottleneck_size:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(self.lr),
                      metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                      min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, batch_size):
        if not tf.test.is_gpu_available:
            print('error no gpu')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training

        self.batch_size = batch_size

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')

        y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
                              return_df_metrics=False)

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)

        df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration)

        keras.backend.clear_session()

        return df_metrics

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test, batch_size=self.batch_size)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time

import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt

class Classifier_RESNET:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, load_weights=False):
        self.output_directory = output_directory
        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            if load_weights == True:
                self.model.load_weights(self.output_directory
                                        .replace('resnet_augment', 'resnet')
                                        .replace('TSC_itr_augment_x_10', 'TSC_itr_10')
                                        + '/model_init.hdf5')
            else:
                self.model.save_weights(self.output_directory + 'model_init.hdf5')
        return

    def build_model(self, input_shape, nb_classes):
        n_feature_maps = 64

        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1

        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL

        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, batch_size):
        if not tf.test.is_gpu_available:
            print('error')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        batch_size = batch_size
        nb_epochs = 10

        mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')

        y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
                              return_df_metrics=False)

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)

        df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration)

        keras.backend.clear_session()

        return df_metrics

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred

In [ ]:
!pip install tensorflow_addons

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import time
from tensorflow.keras import backend as K

#from utils.utils import save_logs
#from utils.utils import calculate_metrics

class Classifier_ENCODER:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False,build=True):
        self.output_directory = output_directory
        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            #self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        # # conv block -1
        # conv1 = keras.layers.Conv1D(filters=32,kernel_size=3,strides=1,padding='same')(input_layer)
        # conv1 = tfa.layers.InstanceNormalization()(conv1)
        # conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
        # conv1 = keras.layers.Dropout(rate=0.2)(conv1)
        # conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
        # # conv block -2
        # conv1 = keras.layers.Conv1D(filters=64,kernel_size=3,strides=1,padding='same')(conv1)
        # conv1 = tfa.layers.InstanceNormalization()(conv1)
        # conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
        # conv1 = keras.layers.Dropout(rate=0.2)(conv1)
        # conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
        # conv block -3
        conv1 = keras.layers.Conv1D(filters=128,kernel_size=5,strides=1,padding='same')(input_layer)
        conv1 = tfa.layers.InstanceNormalization()(conv1)
        conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
        conv1 = keras.layers.Dropout(rate=0.2)(conv1)
        conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
        # conv block -4
        conv2 = keras.layers.Conv1D(filters=256,kernel_size=11,strides=1,padding='same')(conv1)
        conv2 = tfa.layers.InstanceNormalization()(conv2)
        conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
        conv2 = keras.layers.Dropout(rate=0.2)(conv2)
        conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
        # conv block -5
        conv3 = keras.layers.Conv1D(filters=512,kernel_size=21,strides=1,padding='same')(conv2)
        conv3 = tfa.layers.InstanceNormalization()(conv3)
        conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
        conv3 = keras.layers.Dropout(rate=0.2)(conv3)
        # conv block -6
        # conv3 = keras.layers.Conv1D(filters=1024,kernel_size=21,strides=1,padding='same')(conv3)
        # conv3 = tfa.layers.InstanceNormalization()(conv3)
        # conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
        # conv3 = keras.layers.Dropout(rate=0.2)(conv3)
        # split for attention
        attention_data = keras.layers.Lambda(lambda x: x[:,:,:256])(conv3)
        attention_softmax = keras.layers.Lambda(lambda x: x[:,:,256:])(conv3)
        # attention mechanism
        attention_softmax = keras.layers.Softmax(name='attention_vec')(attention_softmax)
        multiply_layer = keras.layers.Multiply()([attention_softmax,attention_data])
        # last layer
        dense_layer = keras.layers.Dense(units=256,activation='sigmoid')(multiply_layer)
        dense_layer = tfa.layers.InstanceNormalization()(dense_layer)
        # output layer
        flatten_layer = keras.layers.Flatten()(dense_layer)
        output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(flatten_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.00001),
                      metrics=['accuracy'])

        file_path = self.output_directory + 'best_model.hdf5'

        # model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path,
        #                                                    monitor='loss', save_best_only=True)

        es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
        #self.callbacks = [model_checkpoint,es]
        self.callbacks = [es]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true):
        if not tf.test.is_gpu_available:
            print('error')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        batch_size = 64
        nb_epochs = 50

        mini_batch_size = batch_size

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

        duration = time.time() - start_time

        #self.model.save(self.output_directory+'last_model.hdf5')

        #model = keras.models.load_model(self.output_directory + 'best_model.hdf5')

        model = self.model

        y_pred = model.predict(x_val)
        print('binary y_pred : ',y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)
        print('argmaxed y_pred : ',y_pred)
        #print(classification_report(np.array(y_val), np.array(y_pred)))

        #save_logs(self.output_directory, hist, y_pred, y_true, duration, lr=False)

        keras.backend.clear_session()
        # return model

    def predict(self, x_test,y_true,x_train,y_train,y_test,return_df_metrics = True):
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            return y_pred
#© 2021 GitHub, Inc.

In [ ]:
# import tensorflow.keras as keras
# import tensorflow as tf
# import tensorflow_addons as tfa
# import numpy as np
# import time

# input_layer = keras.layers.Input((7680,8))


# conv1 = keras.layers.Conv1D(filters=128,kernel_size=5,strides=1,padding='same')(input_layer)
# conv1 = tfa.layers.InstanceNormalization()(conv1)
# conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
# conv1 = keras.layers.Dropout(rate=0.2)(conv1)
# conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
# # conv block -4
# conv2 = keras.layers.Conv1D(filters=256,kernel_size=11,strides=1,padding='same')(input_layer)
# conv2 = tfa.layers.InstanceNormalization()(conv2)
# conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
# conv2 = keras.layers.Dropout(rate=0.2)(conv2)
# conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
# # conv block -5
# conv3 = keras.layers.Conv1D(filters=512,kernel_size=21,strides=1,padding='same')(input_layer)
# conv3 = tfa.layers.InstanceNormalization()(conv3)
# conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
# conv3 = keras.layers.Dropout(rate=0.2)(conv3)
# conv3 = keras.layers.MaxPooling1D(pool_size=2)(conv3)

# concat = tf.keras.layers.concatenate([conv1, conv2, conv3], axis=-1)

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers import Dense, Activation, Multiply, Add, Lambda
from keras.initializers import Constant


class Highway(Layer):

    activation = None
    transform_gate_bias = None

    def __init__(self, activation='relu', transform_gate_bias=-1, **kwargs):
        self.activation = activation
        self.transform_gate_bias = transform_gate_bias
        super(Highway, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        dim = input_shape[-1]
        transform_gate_bias_initializer = Constant(self.transform_gate_bias)
        input_shape_dense_1 = input_shape[-1]
        self.dense_1 = Dense(units=dim, bias_initializer=transform_gate_bias_initializer)
        self.dense_1.build(input_shape)
        self.dense_2 = Dense(units=dim)
        self.dense_2.build(input_shape)
        self.trainable_weightssss = self.dense_1.trainable_weights + self.dense_2.trainable_weights

        super(Highway, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        dim = K.int_shape(x)[-1]
        transform_gate = self.dense_1(x)
        transform_gate = Activation("sigmoid")(transform_gate)
        carry_gate = Lambda(lambda x: 1.0 - x, output_shape=(dim,))(transform_gate)
        transformed_data = self.dense_2(x)
        transformed_data = Activation(self.activation)(transformed_data)
        transformed_gated = Multiply()([transform_gate, transformed_data])
        identity_gated = Multiply()([carry_gate, x])
        value = Add()([transformed_gated, identity_gated])
        return value

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = super().get_config()
        config['activation'] = self.activation
        config['transform_gate_bias'] = self.transform_gate_bias
        return config

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import time

#from utils.utils import save_logs
#from utils.utils import calculate_metrics

class Classifier_ENCODER22:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False,build=True):
        self.output_directory = output_directory
        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            #self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        
        # conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
        conv1 = keras.layers.Conv1D(filters=128,kernel_size=5,strides=1,padding='same')(input_layer)
        conv1 = tfa.layers.InstanceNormalization()(conv1)
        conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
        conv1 = keras.layers.Dropout(rate=0.2)(conv1)
        conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
        # conv block -4
        conv2 = keras.layers.Conv1D(filters=256,kernel_size=11,strides=1,padding='same')(input_layer)
        conv2 = tfa.layers.InstanceNormalization()(conv2)
        conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
        conv2 = keras.layers.Dropout(rate=0.2)(conv2)
        # conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
        conv2 = keras.layers.Conv1D(filters=512,kernel_size=11,strides=1,padding='same')(conv2)
        conv2 = tfa.layers.InstanceNormalization()(conv2)
        conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
        conv2 = keras.layers.Dropout(rate=0.2)(conv2)
        conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
        
        # conv block -5
        # conv3 = keras.layers.Conv1D(filters=128,kernel_size=21,strides=1,padding='same')(input_layer)
        # conv3 = tfa.layers.InstanceNormalization()(conv3)
        # conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
        # conv3 = keras.layers.Dropout(rate=0.2)(conv3)
        # # conv3 = keras.layers.MaxPooling1D(pool_size=2)(conv3)
        # conv3 = keras.layers.Conv1D(filters=256,kernel_size=21,strides=1,padding='same')(conv3)
        # conv3 = tfa.layers.InstanceNormalization()(conv3)
        # conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
        # conv3 = keras.layers.Dropout(rate=0.2)(conv3)
        # conv3 = keras.layers.MaxPooling1D(pool_size=2)(conv3)
        # # conv3 = keras.layers.Conv1D(filters=512,kernel_size=21,strides=1,padding='same')(conv3)
        # conv3 = tfa.layers.InstanceNormalization()(conv3)
        # conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
        # conv3 = keras.layers.Dropout(rate=0.2)(conv3)
        # conv3 = keras.layers.MaxPooling1D(pool_size=2)(conv3)

        concat_r = tf.keras.layers.concatenate([conv1, conv2], axis=-1)
        highway_layer = Highway()(concat_r)
        highway_layer = tfa.layers.InstanceNormalization()(highway_layer)
        highway_layer = keras.layers.Dropout(rate=0.3)(highway_layer)
        highway_layer = keras.layers.MaxPooling1D(pool_size=2)(highway_layer)
        
        attention_data = keras.layers.Lambda(lambda x: x[:,:,:320])(highway_layer)
        attention_softmax = keras.layers.Lambda(lambda x: x[:,:,320:])(highway_layer)
        # attention mechanism
        attention_softmax = keras.layers.Softmax(name='attention_vec')(attention_softmax)
        multiply_layer = keras.layers.Multiply()([attention_softmax,attention_data])
        # last layer
        dense_layer = keras.layers.Dense(units=256,activation='sigmoid')(multiply_layer)
        dense_layer = tfa.layers.InstanceNormalization()(dense_layer)
        # output layer
        flatten_layer = keras.layers.Flatten()(dense_layer)
        output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(flatten_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.00001),
                      metrics=['accuracy'])

        file_path = self.output_directory + 'best_model.hdf5'

        # model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path,
        #                                                    monitor='loss', save_best_only=True)

        es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
        #self.callbacks = [model_checkpoint,es]
        self.callbacks = [es]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, batch_size):
        if not tf.test.is_gpu_available:
            print('error')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        batch_size = batch_size
        nb_epochs = 50

        mini_batch_size = batch_size

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

        duration = time.time() - start_time

        #self.model.save(self.output_directory+'last_model.hdf5')

        #model = keras.models.load_model(self.output_directory + 'best_model.hdf5')

        model = self.model

        y_pred = model.predict(x_val)
        print('binary y_pred : ',y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)
        print('argmaxed y_pred : ',y_pred)
        #print(classification_report(np.array(y_val), np.array(y_pred)))

        #save_logs(self.output_directory, hist, y_pred, y_true, duration, lr=False)

        keras.backend.clear_session()
        # return model

    def predict(self, x_test,y_true,x_train,y_train,y_test,return_df_metrics = True):
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            return y_pred
#© 2021 GitHub, Inc.

In [ ]:


# # keras.layers.MultiHeadAttention()

# input_layer = keras.layers.Input((7680,8))

# # conv block -1
# conv1 = keras.layers.Conv1D(filters=128,kernel_size=5,strides=1,padding='causal')(input_layer)
# conv1 = tfa.layers.InstanceNormalization()(conv1)
# conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
# conv1 = keras.layers.Dropout(rate=0.2)(conv1)
# conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
# # conv block -2
# conv2 = keras.layers.Conv1D(filters=256,kernel_size=11,strides=1,padding='causal')(conv1)
# conv2 = tfa.layers.InstanceNormalization()(conv2)
# conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
# conv2 = keras.layers.Dropout(rate=0.2)(conv2)
# conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
# # conv block -3
# conv3 = keras.layers.Conv1D(filters=512,kernel_size=21,strides=1,padding='causal')(conv2)
# conv3 = tfa.layers.InstanceNormalization()(conv3)
# conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
# conv3 = keras.layers.Dropout(rate=0.2)(conv3)
# # conv block -4
# conv3 = keras.layers.Conv1D(filters=1024,kernel_size=21,strides=1,padding='causal')(conv3)
# conv3 = tfa.layers.InstanceNormalization()(conv3)
# conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
# conv3 = keras.layers.Dropout(rate=0.2)(conv3)


In [ ]:
# conv3.shape

In [ ]:
# # split for attention
# attn_layer = keras.layers.MultiHeadAttention(num_heads=4,key_dim=64, attention_axes=(1, 2))(conv3,conv3)


In [ ]:
# attn_layer.shape

In [ ]:
# # last attn_layer
# dense_layer = keras.layers.Dense(units=256,activation='sigmoid')(attn_layer)
# dense_layer = tfa.layers.InstanceNormalization()(dense_layer)
# # output layer
# flatten_layer = keras.layers.Flatten()(dense_layer)
# output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(flatten_layer)

# model = keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
def fit_classifier(x_train, x_test, y_train, y_test, batch_size):
    #x_train = datasets_dict[dataset_name][0]
    #y_train = datasets_dict[dataset_name][1]
    #x_test = datasets_dict[dataset_name][2]
    #y_test = datasets_dict[dataset_name][3]

    nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

    # save orignal y because later we will use binary
    y_true = np.argmax(y_test, axis=1)

    if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

    input_shape = x_train.shape[1:]
    classifier = create_classifier(classifier_name, input_shape, nb_classes)
    print('classifier_name : ',classifier_name)

    classifier.fit(x_train, y_train, x_test, y_test, y_true, batch_size)


def create_classifier(classifier_name, input_shape, nb_classes, output_directory = '', verbose=True):
    if classifier_name == 'fcn':
        from classifiers import fcn
        return fcn.Classifier_FCN(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'mlp':
        from classifiers import mlp
        return mlp.Classifier_MLP(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'resnet':
        #from classifiers import resnet
        return Classifier_RESNET(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'mcnn':
        from classifiers import mcnn
        return mcnn.Classifier_MCNN(output_directory, verbose)
    if classifier_name == 'tlenet':
        from classifiers import tlenet
        return tlenet.Classifier_TLENET(output_directory, verbose)
    if classifier_name == 'twiesn':
        from classifiers import twiesn
        return twiesn.Classifier_TWIESN(output_directory, verbose)
    if classifier_name == 'encoder':
        #from classifiers import encoder
        return Classifier_ENCODER(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'encoder_lstm':
        #from classifiers import encoder
        return Classifier_ENCODER22(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'cnn':  # Time-CNN
        from classifiers import cnn
        return cnn.Classifier_CNN(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'inception':
        #from classifiers import inception
        return Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose)


# **Denoising Seq AE**

In [ ]:
import scipy.io as sio
import numpy as np

## DEAP

In [ ]:
from tqdm import tqdm
# with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_ICA.pkl', 'rb') as filepath:
#           x_data = pickle.load(filepath)
# with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nzz7680_attn444__l.pkl', 'rb') as filepath:
#           x_data = pickle.load(filepath)
with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nzz7680_attn444__bino_all2.pkl', 'rb') as filepath:
          x_data = pickle.load(filepath)
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
          y_data = pickle.load(filepath)

x_data = x_data.reshape(-1,8064,8)[:,-7680:]#.transpose(0,2,1)
y_data = y_data.reshape(-1,2)[:,1]

y_data[y_data<5] = 0
y_data[y_data>=5] = 1

batch_size = 50


for testSubNo in tqdm(range(1,33)):
  ############################################### main
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')


  indd = testSubNo*40

  X_train1, X_train2, X_test, y_train1, y_train2, y_test = x_data[:indd-40], x_data[indd:], x_data[indd-40:indd], y_data[:indd-40], y_data[indd:], y_data[indd-40:indd]
  y_train = np.concatenate((y_train1,y_train2))
  X_train = np.concatenate((X_train1,X_train2))

  print('X_train : ',X_train.shape)
  print('y_train : ',y_train.shape)
  print('X_test : ',X_test.shape)
  print('y_test : ',y_test.shape)
  ################################################################################
  ################################################################################
  ################################################################################
  # X_train = X_train.reshape(-1,768,32)
  # X_test = X_test.reshape(-1,768,32)

  # y_train_final = np.zeros(10*len(y_train))
  # count = 0

  # for label in y_train:
  #   for i in range(10):
  #     y_train_final[count] = label
  #     count += 1

  # y_test_final = np.zeros(10*len(y_test))
  # count = 0

  # for label in y_test:
  #   for i in range(10):
  #     y_test_final[count] = label
  #     count += 1

  # y_train_final = y_train
  # y_test_final = y_test
  ################################################################################
  ################################################################################
  ################################################################################

  print('test subNo: '+str(testSubNo))

  # change this directory for your machine
  #root_dir = '/b/home/uha/hfawaz-datas/dl-tsc-temp/'

  classifier_name = 'encoder_lstm'
  output_directory = ''

  fit_classifier(X_train, X_test, y_train, y_test, batch_size)

  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')

## Seizure

In [ ]:
from tqdm import tqdm

with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse_5sss_attnn__1ll.pkl', 'rb') as filepath:
          x_data = pickle.load(filepath)
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/y_data_ss_ip.pkl', 'rb') as filepath:
      y_data = pickle.load(filepath)

x_data = x_data.reshape(-1,5120,8)#[:,-7680:]#.transpose(0,2,1)
y_data = y_data.reshape(-1)

# y_data[y_data<5] = 0
# y_data[y_data>=5] = 1

batch_size = 64


for testSubNo in tqdm(range(1,10)):
  ############################################### main
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')


  indd = testSubNo*18

  X_train1, X_train2, X_test, y_train1, y_train2, y_test = x_data[:indd-18], x_data[indd:], x_data[indd-18:indd], y_data[:indd-18], y_data[indd:], y_data[indd-18:indd]
  y_train = np.concatenate((y_train1,y_train2))
  X_train = np.concatenate((X_train1,X_train2))

  print('X_train : ',X_train.shape)
  print('y_train : ',y_train.shape)
  print('X_test : ',X_test.shape)
  print('y_test : ',y_test.shape)
  ################################################################################
  ################################################################################
  ################################################################################
  # X_train = X_train.reshape(-1,768,32)
  # X_test = X_test.reshape(-1,768,32)

  # y_train_final = np.zeros(10*len(y_train))
  # count = 0

  # for label in y_train:
  #   for i in range(10):
  #     y_train_final[count] = label
  #     count += 1

  # y_test_final = np.zeros(10*len(y_test))
  # count = 0

  # for label in y_test:
  #   for i in range(10):
  #     y_test_final[count] = label
  #     count += 1

  # y_train_final = y_train
  # y_test_final = y_test
  ################################################################################
  ################################################################################
  ################################################################################

  print('test subNo: '+str(testSubNo))

  # change this directory for your machine
  #root_dir = '/b/home/uha/hfawaz-datas/dl-tsc-temp/'

  classifier_name = 'encoder_lstm'
  output_directory = ''

  fit_classifier(X_train, X_test, y_train, y_test, batch_size)

  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')

In [ ]:
with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/y_data_ss_ip.pkl', 'rb') as filepath:
      y_data = pickle.load(filepath)

In [ ]:
y_data

## SEED

In [ ]:
from tqdm import tqdm

# with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nz.pkl', 'rb') as filepath:
with open('/content/drive/MyDrive/EMOTION/encoder/DSEQAE_ecoded_dataset_mse88_nz_Seed3_att2.pkl', 'rb') as filepath:
          x_data = pickle.load(filepath)
# with open('/content/drive/MyDrive/EMOTION/VQVAE-trans/Trans-checkpoints-py/labels_val_arousal.pkl', 'rb') as filepath:
#           y_data = pickle.load(filepath)
# temp = sio.loadmat('/content/drive/MyDrive/EMOTION/Preprocessed_EEG/label.mat')
# temp = np.array(temp['label'])[0]
temp = np.array([ 1,  1, 0, 0,  1,  1, 0,  1,  1,  1,  1, 0,  1,  1, 0])
y_data = np.zeros((15,15))

for i in range(15):
  y_data[i] = temp

y_data = y_data.reshape(-1)

x_data = x_data.reshape(-1,30000,16)#.transpose(0,2,1)
# y_data = y_data.reshape(-1,2)[:,1]


for testSubNo in tqdm(range(1,16)):
  ############################################### main
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')


  indd = testSubNo*(15)

  X_train1, X_train2, X_test, y_train1, y_train2, y_test = x_data[:indd-(15)], x_data[indd:], x_data[indd-(15):indd], y_data[:indd-(15)], y_data[indd:], y_data[indd-(15):indd]
  y_train = np.concatenate((y_train1,y_train2))
  X_train = np.concatenate((X_train1,X_train2))

  print('X_train : ',X_train.shape)
  print('y_train : ',y_train.shape)
  print('X_test : ',X_test.shape)
  print('y_test : ',y_test.shape)
  ################################################################################
  ################################################################################
  ################################################################################
  # X_train = X_train.reshape(-1,768,32)
  # X_test = X_test.reshape(-1,768,32)

  # y_train_final = np.zeros(10*len(y_train))
  # count = 0

  # for label in y_train:
  #   for i in range(10):
  #     y_train_final[count] = label
  #     count += 1

  # y_test_final = np.zeros(10*len(y_test))
  # count = 0

  # for label in y_test:
  #   for i in range(10):
  #     y_test_final[count] = label
  #     count += 1

  # y_train_final = y_train
  # y_test_final = y_test
  ################################################################################
  ################################################################################
  ################################################################################

  print('test subNo: '+str(testSubNo))

  # change this directory for your machine
  #root_dir = '/b/home/uha/hfawaz-datas/dl-tsc-temp/'

  classifier_name = 'encoder_lstm'
  output_directory = ''

  fit_classifier(X_train, X_test, y_train, y_test, 8)

  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------')